---

#  🟢 **Minicurso** - Processamento e Visualização de Imagens de Satélite, Relâmpagos, Precipitação, Índices de Vegetação e Queimadas com Python (PyVisSat)

---

> ## **Aula 2:** Analisando Dados de Relâmpagos Estimados por Satélite

---
**OBJETIVO:**

-  Nesta aula prática aprenderemos como trabalhar com os dados de relâmpagos estimado pelo sensor Geostationary Lightning Mapper (GLM) a abordo do satélite [GOES-16](https://space.oscar.wmo.int/satellites/view/goes_16) e [GOES-19](https://space.oscar.wmo.int/satellites/view/goes_19). O objetivo é aprender como plotar e analisar os dados a cada 20s e 5min do GLM e combiná-los com imagens de satélite.

---

**DADOS DE ENTRADA**:
- Serão utilizados os arquivos netCDF a cada 20s fornecidos pela NOAA no repositório da Amazon e a cada 5min reprocessado e fornecido pelo CPTEC/INPE.

    1. $\underline{Amazon\ Web\ Service\ - 20s}$:
        - **Tipo do dado:** pontuais
        - **Formato do dado:** arquivo NETCDF
        - **Nome do arquivo:** OR_GLM-L2-LCFA_G16_s20210010306000_e20210010306204_c20210010306215.nc
        - **Fonte dos dados:** FTP da Amazon para [GOES-16](https://noaa-goes16.s3.amazonaws.com/index.html#GLM-L2-LCFA/) e [GOES-19](https://noaa-goes19.s3.amazonaws.com/index.html#GLM-L2-LCFA/).


    2. $\underline{CPTEC/INPE - 5min}$:
        - **Tipo do dado:** matriz de 1664 linhas x 1664 colunas  
        - **Formato do dado:** arquivo NETCDF
        - **Nome do arquivo:** S11635949_202201141500.nc
        - **Fonte dos dados:** FTP do CPTEC/INPE para [GOES-16](http://ftp.cptec.inpe.br/goes/goes16/goes16_web/glm_acumulado_nc/) e [GOES-19](http://ftp.cptec.inpe.br/goes/goes19/goes19_web/glm_acumulado_nc/)


---

**DADOS DE SAÍDA:**
- Serão produzidas figuras no formato JPG, mapas interativos e animações de imagens em formato GIF.

---

**PROCEDIMENTO REALIZADO:**
- Os seguintes procedimentos são realizados nessa aula:

    1.   **1° Passo:** Instalação das Bibiotecas
    2.   **2° Passo:** Download de Arquivos Auxiliares
    3.   **3° Passo:** Declarando Funções
    4.   **PARTE 1):** Dados da Amazon a cada 20s - Download e Conhecendo os Dados
    5.   **PARTE 2):** Dados da Amazon a cada 20s - Visualização da Imagem de um Arquivo de Relâmpago num Mapa Estático
    6.   **PARTE 3):** Dados da Amazon a cada 20s - Visualização da Imagem de Vários Arquivos de Relâmpagos num Mapa Estático
    7.   **PARTE 4):** Dados da Amazon a cada 20s - Visualização da Imagem de Vários Arquivos de Relâmpagos num Mapa Interativo
    8.   **PARTE 5):** Dados da Amazon a cada 20s - Acumulando Relâmpagos para um Dia de Dados
    9.  **PARTE 6):** Dados da Amazon a cada 20s - Combinação de Imagens de Satélite e Relâmpagos
    10.  **PARTE 7):** Dados da Amazon a cada 20s - Evolução Temporal da Temperatura de Brilho do IR e Relâmpagos
    11. **PARTE 8):** Dados do INPE a cada 5min - Download e Processamento
    12. **PARTE 9):** Dados do INPE a cada 5min - Visualização de Mapa de Relâmpago para um Determinado Dia
    13. **PARTE 10):** Dados do INPE a cada 5min - Visualização de Série Temporal
---
**OBSERVAÇÕES IMPORTANTES**:
1. Necessário possuir uma conta do Gmail.
2. Salvar o código no seu drive. Para isto clicar em `Arquivo` e depois em `Salvar uma cópia no drive` e fazer o login numa conta google.

---
**REALIZADO E MINISTRADO POR:**
- Prof. Dr. Enrique Vieira Mattos - UNIFEI: enrique@unifei.edu.br / https://github.com/evmpython
- Atualizado em: 09/03/2026
 ---

![Texto alternativo](https://github.com/evmpython/Minicurso_UFMS_SEMADESC_marco_2026/blob/main/04_logos/logo-aula02.png?raw=true)

# **1° Passo:** Instalação das Bibliotecas

O Google Colaboratory (mais conhecido como Colab ) roda em uma máquina Linux na Nuvem do Google, e esta máquina já possui o Python instalado. Ou seja, todos os comandos do Linux que você já conhece são válidos nas células de código, bastando apenas você inserir um ponto de exclamação `!` no início de cada célula de código. Por exemplo, vamos verificar a versão do Python que temos disponível no Google Colab neste momento.

In [ ]:
# vamos verificar a versão do python instalado no Google Colab
!python --version

Nosso primeiro passo será instalar as bibliotecas necessárias (e suas dependências) para a execução dos códigos. Inicialmente precisaremos instalar as seguintes bibliotecas:

*   `Netcdf4:` Ler os dados de arquivos no formato NetCDF
*   `Cartopy:` Adicionar mapas aos plots
*   `Boto3:` Download de dados GOES-16/19 diretamente da nuvem da Amazon Web Services (AWS)
*   `geobr:` Trabalhar com shapefiles de municípios e estados
*   `salem:` Extrair dados de um shapefile
*   `rasterio, pyproj, geopandas e descartes:` Extrair informações de uma região


In [ ]:
# verificando as bibliotecas instaladas no Colab
!pip list

In [ ]:
# verificando se a biblioteca "boto3" esta instalada no Colab
!pip show boto3

Veja que a mensagem acima indica que a biblioteca boto3 não esta instalada no Colab. Então iremos proceder com a instalação das bibliotecas que iremos utilizar e que ainda não estejam instaladas no Colab.

In [ ]:
# instalando as bibliotecas
!pip install -q netcdf4 ultraplot cartopy boto3 geobr salem rasterio pyproj geopandas descartes

In [ ]:
# verificando se a biblioteca "boto3" foi instalada no Colab
!pip show boto3

# **2° Passo:** Download de Arquivos Auxiliares



Neste passo vamos fazer o download de alguns arquivos auxiliares necessários para os códigos que serão utilizados nesta aula do curso. Os arquivos estão disponíveis no [GitHub](https://github.com/evmpython/Minicurso_UFMS_SEMADESC_marco_2026/tree/main/01_utils) do nosso curso:

1. `utilities_goes16e19.py:` Código python que contêm algumas funções para o processamento de dados do satélite GOES-16 e GOES-19.
2. `ir.cpt:` Paleta de cores para o canal do infravermelho do satélite GOES-16 e GOES-19.

In [ ]:
import time
# download do arquivo "utilities_goes16e19.py"
!wget -c https://github.com/evmpython/Minicurso_UFMS_SEMADESC_marco_2026/raw/main/01_utils/utilities_goes16e19.py

# download da paleta de cores para o canal do infravermelho
!wget -c https://github.com/evmpython/Minicurso_UFMS_SEMADESC_marco_2026/raw/main/01_utils/ir.cpt

# **3° Passo:** Declarando Funções

In [ ]:
#==================================================================================================#
#          FUNÇÃO DE BAIXA OS DADOS DO GLM DE 5MIN DO CPTEC/INPE
#==================================================================================================#
def download_GLM5min_CPTEC(ano, mes, dia, hor, min, goes_number, dir_input):

    ''' Função para baixar os arquivos netcdf de 5min do GLM processado e disponibilizado pelo CPTEC/INPE

    Parâmetros:
               ano (string): ano do arquivo
               mes (string): mês do arquivo
               dia (string): dia do arquivo
               hor (string): hora do arquivo
               min (string): minuto do arquivo
               goes_number (string): número do satélite GOES: 16 ou 19
               dir_input (string): ano do diretório onde deverá ser salvo o arquivo depois de baixado

    Retorna:
            file_glm_download (string): nome do diretório + nome do arquivo que foi baixado
            codigo_produto (string): código do produto: GOES-16 (S11635949) e GOES-19 (S11162307)
    '''

    print('.... Baixando dado ===>', 'DATA=', ano, mes, dia, hor, min)

    # caminho do ftp do cptec
    if goes_number == '16':
        ftp_cptec_glm_5min = 'http://ftp.cptec.inpe.br/goes/goes16/goes16_web/glm_acumulado_nc/'
        codigo_produto = 'S11635949'

    elif goes_number == '19':
        ftp_cptec_glm_5min = 'http://ftp.cptec.inpe.br/goes/goes19/goes19_web/glm_acumulado_nc/'
        codigo_produto = 'S11162307'

    # nome do diretório + nome do arquivo
    # exemplo: "http://ftp.cptec.inpe.br/goes/goes19/goes19_web/glm_acumulado_nc/2025/11/S11162307_202511130330.nc"
    file_glm5min_cptec =  f'{ftp_cptec_glm_5min}{ano}/{mes}/{codigo_produto}_{ano}{mes}{dia}{hor}{min}.nc'

    # baixando o arquivo
    !wget {file_glm5min_cptec} -P {dir_input}

    # local da imagem que foi baixada
    file_glm_download = f'{dir_input}/{codigo_produto}_{ano}{mes}{dia}{hor}{min}.nc'

    return file_glm_download, codigo_produto

#==================================================================================================#
#                   FUNÇÃO QUE FAZ O DOWNLOAD DOS DADOS DE 20S DA AWS
#==================================================================================================#
def download_GLM20s_AWS(yyyymmddhhmnss, goes_number, path_dest):

    ''' Função para baixar os arquivos netcdf de 20s do GLM processado e disponibilizado pela Amazon Web Service (AWS)

    Parâmetros:
               yyyymmddhhmnss (string): ano+mes+dia+hora+minuto+segundo. Exemplo: 20251113033020
               goes_number (string): número do satélite GOES: 16 ou 19
               path_dest (string): nome do direório + nome do arquivo que foi baixado

    Retorna:
            file_name (string): nome do direório + nome do arquivo que foi baixado

    Observação: acesso aos dados
                    GOES-16: https://noaa-goes16.s3.amazonaws.com/index.html#ABI-L2-CMIPF/
                    GOES-19: https://noaa-goes19.s3.amazonaws.com/index.html#ABI-L2-CMIPF/
    '''

    year = datetime.strptime(yyyymmddhhmnss, '%Y%m%d%H%M%S').strftime('%Y')
    day_of_year = datetime.strptime(yyyymmddhhmnss, '%Y%m%d%H%M%S').strftime('%j')
    hour = datetime.strptime(yyyymmddhhmnss, '%Y%m%d%H%M%S').strftime('%H')
    min = datetime.strptime(yyyymmddhhmnss, '%Y%m%d%H%M%S').strftime('%M')
    seg = datetime.strptime(yyyymmddhhmnss, '%Y%m%d%H%M%S').strftime('%S')

    # informação do repositório da AMAZON. Exemplo: 'noaa-goes19'
    bucket_name = f'noaa-goes{goes_number}'

    # inicializa o S3 cliente
    s3_client = boto3.client('s3', config=Config(signature_version=UNSIGNED))

    # estutura do arquivo. Exemplo: "GLM-L2-LCFA/2025/317/03/OR_GLM-L2-LCFA_G19_s20253170330200"
    product_name = "GLM-L2-LCFA"
    prefix = f'{product_name}/{year}/{day_of_year}/{hour}/OR_{product_name}_G{goes_number}_s{year}{day_of_year}{hour}{min}{seg}'

    # pesquisa pelo arquivo no servidor
    s3_result = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=prefix, Delimiter = "/")

    # verifica se o arquivo esta diponível
    #-----------------------------------------------------------------------------------------------------------
    if 'Contents' not in s3_result:

        # se o arquivo não esta disponível
        print(f'No files found for the date: {yyyymmddhhmnss}, Product-{product_name}')
        return -1
    else:

        # se o arquivo existe
        for obj in s3_result['Contents']:
            key = obj['Key']

            # imprime o nome do arquivo
            file_name = key.split('/')[-1].split('.')[0]

        # baixa o arquivo
        if os.path.exists(f'{path_dest}/{file_name}.nc'):
            print(f'File {path_dest}/{file_name}.nc exists')
        else:
            print(f'Downloading file {path_dest}/{file_name}.nc')
            s3_client.download_file(bucket_name, key, f'{path_dest}/{file_name}.nc')

    return f'{file_name}'

#==================================================================================================#
#                     FUNÇÃO QUE CALCULA O ÍNDICE "I" e "J" DA LOCALIZAÇÃO DO FLASH
#==================================================================================================#
def index(longitudes_matriz, latitudes_matriz, lon_raio, lat_raio):

    ''' Função para calcular o índice (i e j) do pixel de uma matriz que o relâmpago pertence

    Parâmetros:
               longitudes_matriz (array): array de uma dimensão das longitudes da matriz em graus
               latitudes_matriz (array): array de uma dimensão das latitudes da matriz em graus
               lon_raio (float): valor da longitude do relâmpago em graus
               lat_raio (float): valor da latitude do relâmpago em graus

    Retorna:
            indice_lat_raio (float): índice da latitude (ou seja, da linha) do pixel da matriz que o relâmpago pertence
            indice_lon_raio (float): índice da longitude (ou seja, da coluna) do pixel da matriz que o relâmpago pertence
    '''

    # calcula a diferença entre as lats/lons da matriz e a latitude/longitude do relâmpago
    distancia_lon = (longitudes_matriz - lon_raio)**2
    distancia_lat = (latitudes_matriz - lat_raio)**2

    # índice da longitude e latitude do relâmpago
    indice_lon_raio = np.nonzero(distancia_lon == np.min(distancia_lon))
    indice_lat_raio = np.nonzero(distancia_lat == np.min(distancia_lat))

    # retorna os valores dos índices calculados
    return indice_lat_raio, indice_lon_raio

#**PARTE 1)**: `Dados da Amazon a cada 20s` - Download e Conhecendo os Dados

Nesta parte da nossa aula iremos aprender como fazer o download dos dados de `ocorrências de flashes (relâmpagos)` do sensor GLM abordo dos satélites GOES-16 e GOES-19. Os dados de relâmpagos são processados e disponibilizados pela NOAA no FTP da Amazon para o [GOES-16](https://noaa-goes16.s3.amazonaws.com/index.html#GLM-L2-LCFA/) e [GOES-19](https://noaa-goes19.s3.amazonaws.com/index.html#GLM-L2-LCFA/). Estes dados estão em arquivos netCDF a cada 20s, contendo as ocorrências de flashes, grupos e eventos nas Américas, e outras informações.

Exemplo: `OR_GLM-L2-LCFA_G19_s20253170330200_e20253170330400_c20253170330415.nc`

- `OR_GLM-L2-LCFA_G16`:
    1.    `OR`: Significa On-Readout. Indica que os dados são processados a partir da transmissão direta do satélite em tempo real.

    2.   `GLM`: Significa *Geostationary Lightning Mapper*. É o nome do instrumento que coletou os dados. O GLM é o primeiro instrumento de imageamento de relâmpagos em órbita geoestacionária, projetado para detectar continuamente a totalidade dos relâmpagos (nuvem-nuvem e nuvem-solo) sobre as Américas e oceanos adjacentes.

    3.   `L2`: Significa Level 2. Indica que são dados de nível 2, ou seja, um produto geofísico derivado. Os dados brutos (nível 1b) foram processados para gerar informações sobre a localização e energia dos eventos de relâmpagos.

    4.   `LCFA`: Significa *Lightning Cluster Filter Algorithm*. Este é o produto principal do GLM. O "LCFA" é o algoritmo que agrupa os pixels individuais de luz detectados (chamados "events") em aglomerados significativos, que representam: Events: A detecção bruta de um único pixel em um instante. Groups: Um aglomerado de "events" no mesmo flash que ocorrem quase simultaneamente. Flashes: A agregação completa de "groups" que constituem um único flash de relâmpago.

    4.   `G19`: Significa GOES-19. Satélite de origem dos dados (GOES-East).

- `s20253170330200_e20253170330400_c20253170330415.nc`:

    1. s20253170330200 - Tempo de Início (Start Time)
        - 2025: Ano (2025)
        - 317: Dia Juliano (317 = 13 de Novembro)
        - 0330200: Hora, Minuto, Segundo (03:20:20.0 UTC)
        - Traduzindo: A varredura da imagem começou em 13 de Novembro de 2025 às 03:20:20.0 UTC.

    2.  e20253170330400 - Tempo de Término (End Time)
        - 2025: Ano (2025)
        - 317: Dia Juliano (317 = 13 de Novembro)
        - 0330400: Hora, Minuto, Segundo (03:30:40.0 UTC)
        - Traduzindo: A varredura da imagem terminou em 13 de Novembro de 2025 às 03:20:40.0 UTC.

    3. c20253170330415 - Tempo de Criação (Creation Time)
        - 2025: Ano (2025)
        - 317: Dia Juliano (317 = 13 de Novembro)
        - 0330415: Hora, Minuto, Segundo (03:30:41.5 UTC)
        - Traduzindo: O arquivo de dados final foi gerado e fechado no sistema em 13 de Novembro de 2025 às 03:20:41.5 UTC.
---


`1. Downloado do arquivo:` neste exemplo iremos faz o download do arquivo do dia 13 de novembro de 2025 às 03 horas e 30 minutos e 20 segundos. A definição desta data/horário esta declarada na variável `yyyymmddhhmnss = '20251113033020'`. Os dados de relâmpagos são baixados através da função `download_GLM20s_AWS` que declaramos anteriormente no 3° Passo desta aula.

In [ ]:
#========================================================================================================================#
#                                          IMPORTAÇÃO DAS BIBLIOTECAS
#========================================================================================================================#
import xarray as xr
import numpy as np
from datetime import timedelta, datetime
import os
import boto3
from botocore import UNSIGNED
from botocore.config import Config

#========================================================================================================================#
#                                         CRIA DIRETÓRIO DE ENTRADA
#========================================================================================================================#
dir_input = "/content/input/Parte_1"; os.makedirs(dir_input, exist_ok=True)

#========================================================================================================================#
#                                          DEFINE A DATA DO ARQUIVO
#========================================================================================================================#
yyyymmddhhmnss = '20251113033020' # ano(2025), mês(11), dia(13), hora(03), minuto(30) e segundos(20)

#========================================================================================================================#
#                                          DEFINE SE GOES-16 OU GOES-19
#========================================================================================================================#
start_g19 = datetime(2025,4,7,0,0)
imagem_atual = datetime.strptime(yyyymmddhhmnss, '%Y%m%d%H%M%S')
goes_number = '16' if imagem_atual < start_g19 else '19'

#========================================================================================================================#
#                                          FAZ O DOWNLOAD DO ARQUIVO
#========================================================================================================================#
file_glm20s = download_GLM20s_AWS(yyyymmddhhmnss, goes_number, dir_input)

Veja que o arquivo foi baixado para o `/content/` da sua sessão do Colab, conforme indicado na figura a seguir:

![Texto alternativo](https://github.com/evmpython/Minicurso_UFMS_SEMADESC_marco_2026/blob/main/04_logos/glm_content.png?raw=true)

`2. Leitura do arquivo:` para leitura do arquivo iremos utilizar a função [open_dataset](https://docs.xarray.dev/en/stable/generated/xarray.open_dataset.html) da biblioteca xarray. Veja como é simples! Em uma única linha fizemos a leitura do arquivo NetCDF.

In [ ]:
# leitura do arquivo
glm_20s = xr.open_dataset(f'{dir_input}/{file_glm20s}.nc')

`3. Exibindo os dados:` a seguir iremos entender um pouco sobre os dados e as informações que estes arquivos de relâmpagps contêm.

Com os dados disponíveis na variável `glm_20s`, o `Xarray` interpreta-os como um objeto chamado de `Dataset`. Veja o resumo da figura a seguir:

![Texto alternativo](https://github.com/evmpython/Minicurso_UFMS_SEMADESC_marco_2026/blob/main/04_logos/xarray.png?raw=true)

In [ ]:
# mostra o tipo dos dados
type(glm_20s)

* Arquivos em netcdf são auto-descritivos, o que significa que eles possuem uma grande quantidade de **meta-dados** e informações sobre eles dentro de sua própria estrutura.

* O `Xarray` consegue mostrar todas essas informações de uma maneira bem interativa no Colab.

In [ ]:
# mostra os dados
glm_20s

In [ ]:
# mostrando as VARIÁVEIS dos dados
glm_20s.data_vars

In [ ]:
# mostrando as COORDENADAS dos dados
glm_20s.coords

In [ ]:
# mostrando os ATRIBUTOS
glm_20s.attrs.items()

In [ ]:
# LATITUDE dos flashes
glm_20s['flash_lat']

In [ ]:
# transformando a latitude dos flashes para um array
lats_flash = np.array((glm_20s['flash_lat']))
lats_flash

In [ ]:
# formato do array
lats_flash.shape

In [ ]:
# TEMPO da imagem
print(glm_20s['product_time'].values)

In [ ]:
# data/horário do PRIMEIRO evento do flash
print('Data/horário do PRIMEIRO evento do flash = ',  glm_20s['flash_time_offset_of_first_event'][0].values) # GLM L2+ Lightning Detection: time of occurrence of first constituent event in flash

# data/horário do ÚLTIMO evento do flash
print('Data/horário do ÚLTIMO evento do flash = ',  glm_20s['flash_time_offset_of_last_event'][0].values) # GLM L2+ Lightning Detection: time of occurrence of last constituent event in flash

In [ ]:
# ÁREA (m2) dos flashes
glm_20s['flash_area']

In [ ]:
# ENERGIA (Joules) dos flashes
glm_20s['flash_energy']

In [ ]:
# FLAG DE QUALIDADE dos flashes
glm_20s['flash_quality_flag']

Agora que conhecemos um pouco sobre os dados do sensor GLM fornecidos pela Amazon Web Service (AWS), iremos plotar alguns gráficos com estes dados. Então bora para a próxima etapa! 🚀

#**PARTE 2)**: `Dados da Amazon a cada 20s` - Visualização da Imagem de um Arquivo de Relâmpago num Mapa Estático

Nesta etapa da aula iremos plotar a imagem dos flashes que ocorreram na área total do GLM. Iremos plotar os flashes do arquivo do dia **13 de novembro de 2025 ás 03 horas e 20 minutos e 20 segundos UTC**. Os flashes deste arquivo são aqueles que ocorreram entre às `03h20min20s` e `03h20min40s`, ou seja num intervalo de tempo de 20s. Para isto iremos fazer o mesmo procedimento de baixar e realizar a leitura dos dados que realizamos na Parte 1 da aula, e depois iremos plotar o mapa com a biblioteca ultraplot.

`Em resumo realizaremos o seguinte passo-a-passo:`

1.   **Importação das bibliotecas**
2.   **Criação do diretório de entrada e saída**
3.   **Definição da data e horário**
4.   **Definição se GOES-16 ou GOES-19**
5.   **Download do arquivo do GLM**
6.   **Leitura do arquivo do GLM**
7.   **Plota figura com o ultraplot**


In [ ]:
#========================================================================================================================#
#                                           IMPORTAÇÃO DAS BIBLIOTECAS
#========================================================================================================================#
import xarray as xr
import numpy as np
from datetime import timedelta, datetime
import os
import boto3
from botocore import UNSIGNED
from botocore.config import Config
import ultraplot as uplt
import cartopy.crs as ccrs
import warnings
warnings.filterwarnings('ignore')

#========================================================================================================================#
#                                          CRIA DIRETÓRIO DE ENTRADA E SAÍDA
#========================================================================================================================#
dir_input = "/content/input/Parte_2"; os.makedirs(dir_input, exist_ok=True)
dir_output = "/content/output/Parte_2"; os.makedirs(dir_output, exist_ok=True)

#========================================================================================================================#
#                                          DEFINE A DATA DO ARQUIVO
#========================================================================================================================#
# define a data/horário
yyyymmddhhmnss = '20251113033020' # ano(2025), mês(11), dia(13), hora(03), minuto(30) e segundos(20)

# extrai o ano, mês, dia, hora, minuto e segundos
yyyy = datetime.strptime(yyyymmddhhmnss, '%Y%m%d%H%M%S').strftime('%Y')
mm = datetime.strptime(yyyymmddhhmnss, '%Y%m%d%H%M%S').strftime('%m')
dd = datetime.strptime(yyyymmddhhmnss, '%Y%m%d%H%M%S').strftime('%d')
hh = datetime.strptime(yyyymmddhhmnss, '%Y%m%d%H%M%S').strftime('%H')
mn = datetime.strptime(yyyymmddhhmnss, '%Y%m%d%H%M%S').strftime('%M')
ss = datetime.strptime(yyyymmddhhmnss, '%Y%m%d%H%M%S').strftime('%S')

#========================================================================================================================#
#                                            DEFINE SE GOES-16 OU GOES-19
#========================================================================================================================#
# o satélite GOES-16 esteve em operação entre 2017 e 06/abril/2025, depois desta data é o satélite GOES-19
start_g19 = datetime(2025,4,7,0,0)
imagem_atual = datetime.strptime(yyyymmddhhmnss, '%Y%m%d%H%M%S')
goes_number = '16' if imagem_atual < start_g19 else '19'

#========================================================================================================================#
#                                            FAZ O DOWNLOAD DO ARQUIVO
#========================================================================================================================#
file_glm20s = download_GLM20s_AWS(yyyymmddhhmnss, goes_number, dir_input)

#========================================================================================================================#
#                                          LEITURA DO ARQUIVO
#========================================================================================================================#
# leitura do arquivo
ds = xr.open_dataset(f'{dir_input}/{file_glm20s}.nc')

# inserindo as latitudes e longitudes num array do python (matriz)
lats_flash, lons_flash = np.array((ds['flash_lat'])), np.array((ds['flash_lon']))

# limites da imagem para a América
lonmin_america, lonmax_america, latmin_america, latmax_america = -141.6, -8.52, -66.56, 66.48

#========================================================================================================================#
#                                               PLOTA FIGURA
#========================================================================================================================#
# moldura da figura
fig, ax = uplt.subplots(figsize=(5.5,5), tight=True, proj='pcarree')

# formatação dos eixos da figura
ax.format(coast=True, borders=True, innerborders=False,
          labels=True, latlines=20, lonlines=20,
          latlim=(latmin_america, latmax_america), lonlim=(lonmin_america, lonmax_america),
          title=f'GOES-{goes_number} GLM Flashes\n{yyyy}-{mm}-{dd} às {hh}:{mn}:{ss} UTC')

# plota localização dos flashes
ax.scatter(lons_flash,
           lats_flash,
           transform=ccrs.PlateCarree(),
           marker='o', s=10, facecolor='bright red', edgecolor='black',
           linewidth=1,
           alpha=0.8,
           label=f'Flashes={str(len(lats_flash)).zfill(4)}')

# adiciona legenda
ax.legend(ncol=1, loc='lr', frameon=True)

# salva figura
fig.save(f'{dir_output}/Parte_2_G{goes_number}_GLM_flash_{yyyy}-{mm}-{dd}_{hh}:{mn}:{ss}.png')

# exibe na tela
uplt.show()

---
>`EXERCÍCIO DE FIXAÇÃO:`
1.   Mudar a data/horário.
---

#**PARTE 3)**: `Dados da Amazon a cada 20s` - Visualização da Imagem de Vários Arquivos de Relâmpagos num Mapa Estático

Nesta etapa da aula iremos plotar os flashes que ocorreram num intervalo de 2 horas. Como referência iremos usar o horário atual do relógio e determinaremos o horário 120min antes. Assim, separaremos e acumularemos os flashes em quatro intervalos de tempos:

1.   `00 e 30min:` flashes que ocorreram entre o horário atual do relógio e 30 minutos antes.

2.   `30 e 60min:` flashes que ocorreram entre 30 minutos antes e 60 minutos antes do horário atual do relógio.

3.   `60 e 90min:`  flashes que ocorreram entre 60 minutos antes e 90 minutos antes do horário atual do relógio.

4.   `90 e 120min:`  flashes que ocorreram entre 90 minutos antes e 120 minutos antes do horário atual do relógio.

Este tipo de análise é muito importante pois possibilita avaliarmos o deslocamento dos relâmpagos nas últimas 2horas. Possibilitando assim, saber para qual direção os flashes estão se deslocando e se a quantidade de relâmpagos esta aumentando ou diminuindo ao longo das últimas 2 horas.


## Download dos Dados:



Nesta etapa iremos realizar o download dos dados do GLM que são a cada 20s. O download de 2 horas de dados demora aproximadamente `5min`.

In [ ]:
%%time
#========================================================================================================================#
#                                          IMPORTAÇÃO DAS BIBLIOTECAS
#========================================================================================================================#
from datetime import timedelta, datetime
import os
import boto3
from botocore import UNSIGNED
from botocore.config import Config

#========================================================================================================================#
#                                        CRIA DIRETÓRIO DE ENTRADA E SAÍDA
#========================================================================================================================#
dir_input = "/content/input/Parte_3"; os.makedirs(dir_input, exist_ok=True)
dir_output = "/content/output/Parte_3"; os.makedirs(dir_output, exist_ok=True)

#========================================================================================================================#
#                                          DEFINE A DATA DO ARQUIVO
#========================================================================================================================#
# OPÇÃO-1: você pode definir a data inicial e final dos flashes. Lembrando que dever ser sempre um intervalo de 120min.
#anoi, mesi, diai, hori, mini = '2025', '11', '13', '03', '30'
#anof, mesf, diaf, horf, minf = '2025', '11', '13', '05', '30'

# OPÇÃO-2: o código automaticamente extrai a data/horário atual do relógio e determina a data/horário depois de 120min.
agora = datetime.now()
anof, mesf, diaf, horf, minf = str(agora.year), str(agora.month).zfill(2), str(agora.day).zfill(2), str(agora.hour).zfill(2), str(agora.minute).zfill(2)

# calcula a data/hora de 120 minutos atrás
data_120min_atras = datetime.now() - timedelta(minutes=120)

# extrai o ano, mês, dia, hora, minuto inicial (ou seja, 120 min antes da data atual)
anoi = str(data_120min_atras.year)
mesi = str(data_120min_atras.month).zfill(2)
diai = str(data_120min_atras.day).zfill(2)
hori = str(data_120min_atras.hour).zfill(2)
mini = str(data_120min_atras.minute).zfill(2)

#========================================================================================================================#
#                                            DOWNLOAD DOS DADOS
#========================================================================================================================#
# coloca a data INICIAL numa string
date_ini = str(datetime(int(anoi),int(mesi),int(diai),int(hori),int(mini))) # Exemplo: 2026-03-01 19:16:00

# coloca a data FINAL numa string
date_end = str(datetime(int(anof),int(mesf),int(diaf),int(horf),int(minf))) # Exemplo: 2026-03-01 21:16:00

# data inicial é fixada como a data do loop
date_loop = date_ini

# loop nos arquivos do GLM
while (date_loop < date_end):

    # data
    yyyymmddhhmnss = datetime.strptime(date_loop, '%Y-%m-%d %H:%M:%S').strftime('%Y%m%d%H%M%S')

    # extrai o ano, mês, dia, hora, minuto e segundos
    ano = datetime.strptime(date_loop, '%Y-%m-%d %H:%M:%S').strftime('%Y')
    mes = datetime.strptime(date_loop, '%Y-%m-%d %H:%M:%S').strftime('%m')
    dia = datetime.strptime(date_loop, '%Y-%m-%d %H:%M:%S').strftime('%d')
    hor = datetime.strptime(date_loop, '%Y-%m-%d %H:%M:%S').strftime('%H')
    min = datetime.strptime(date_loop, '%Y-%m-%d %H:%M:%S').strftime('%M')
    seg = datetime.strptime(date_loop, '%Y-%m-%d %H:%M:%S').strftime('%S')

    # define se GOES-16 ou GOES-19
    start_g19 = datetime(2025,4,7,0,0)
    imagem_atual = datetime.strptime(yyyymmddhhmnss, '%Y%m%d%H%M%S')
    goes_number = '16' if imagem_atual < start_g19 else '19'
    print(f'DOWNLOAD DO ARQUIVO GLM ===>>> {ano}-{mes}-{dia} {hor}:{min}:{seg}')

    # download o arquivo
    file_glm20s = download_GLM20s_AWS(yyyymmddhhmnss, goes_number, dir_input)

    # incrementa a variável the date_loop (ou seja, soma 20s)
    date_loop = str(datetime.strptime(date_loop, '%Y-%m-%d %H:%M:%S') + timedelta(seconds=20))
    print('\n')

## Plota figura

In [ ]:
#========================================================================================================================#
#                                           IMPORTAÇÃO DAS BIBLIOTECAS
#========================================================================================================================#
import xarray as xr
import numpy as np
import pandas as pd
import ultraplot as uplt
import cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader
import glob
import warnings
warnings.filterwarnings('ignore')

#========================================================================================================================#
#                                        CRIA DIRETÓRIO DE ENTRADA E SAÍDA
#========================================================================================================================#
dir_input = "/content/input/Parte_3"; os.makedirs(dir_input, exist_ok=True)
dir_output = "/content/output/Parte_3"; os.makedirs(dir_output, exist_ok=True)

#========================================================================================================================#
#                                              PROCESSAMENTO
#========================================================================================================================#
# limites da imagem para América
lonmin_america, lonmax_america, latmin_america, latmax_america = -141.6, -8.52, -66.56, 66.48

# lista dos arquivos que foram baixados
files = sorted(glob.glob(f'{dir_input}/*nc'))

# inicializa os arrays de time, latitude e longitude dos flashes
times_flash = []
lats_flash = np.array([], dtype=np.float32)
lons_flash = np.array([], dtype=np.float32)

# loop nos arquivos
for file in files:

    # leitura do arquivo
    glm_20s = xr.open_dataset(file)

    # appenda as times/lats/longs
    times_flash.extend(glm_20s['flash_time_offset_of_first_event'].values) # data/horário do primeiro evento do flash
    lats_flash = np.append(lats_flash, glm_20s['flash_lat'].values) # data/horário da latitude do flash
    lons_flash = np.append(lons_flash, glm_20s['flash_lon'].values) # data/horário da longitude do flash

    # fecha o arquivo
    glm_20s.close()

# coloca os flashes num dataframe
data_flash = {'time': times_flash, 'lat': lats_flash, 'lon': lons_flash}
df_flash = pd.DataFrame(data_flash)

# tranforma a coluna data para índice do dataframe
df_flash.set_index('time', inplace=True)

# ordena o dataframe
df_flash.sort_index(inplace=True)

#========================================================================================================================#
#                                            SEPARA OS DADOS POR CLASSE DE TEMPO
#========================================================================================================================#
# data atual
dt_data_ultima = datetime(int(anof), int(mesf), int(diaf), int(horf), int(minf), 0) # transforma a data atual para formato de datas

# 0-30min: intervalo entre (data_ultima - 30 min) e data_ultima
dt_time_interval_end_0_30min = dt_data_ultima
dt_time_interval_start_0_30min = dt_data_ultima - timedelta(minutes=30)
time_1a = str(dt_time_interval_end_0_30min)
time_1b = str(dt_time_interval_start_0_30min)

# 30-60min: intervalo entre (data_ultima - 60 min) e (data_ultima - 30 min)
dt_time_interval_end_30_60min = dt_time_interval_start_0_30min
dt_time_interval_start_30_60min = dt_data_ultima - timedelta(minutes=60)
time_2a = str(dt_time_interval_end_30_60min)
time_2b = str(dt_time_interval_start_30_60min)

# 60-90min: interval entre (data_ultima - 90 min) e (data_ultima - 60 min)
dt_time_interval_end_60_90min = dt_time_interval_start_30_60min
dt_time_interval_start_60_90min = dt_data_ultima - timedelta(minutes=90)
time_3a = str(dt_time_interval_end_60_90min)
time_3b = str(dt_time_interval_start_60_90min)

# 90-120min: interval entre (data_ultima - 120 min) e (data_ultima - 90 min)
dt_time_interval_end_90_120min = dt_time_interval_start_60_90min
dt_time_interval_start_90_120min = dt_data_ultima - timedelta(minutes=120)
time_4a = str(dt_time_interval_end_90_120min)
time_4b = str(dt_time_interval_start_90_120min)

# seleciona os flashes de cada classe de tempo
df_00_to_30min = df_flash[(df_flash.index > time_1b) & (df_flash.index <= time_1a)]
df_30_to_60min = df_flash[(df_flash.index > time_2b) & (df_flash.index <= time_2a)]
df_60_to_90min = df_flash[(df_flash.index > time_3b) & (df_flash.index <= time_3a)]
df_90_to_120min = df_flash[(df_flash.index > time_4b) & (df_flash.index <= time_4a)]

#========================================================================================================================#
#                                                 PLOTA FIGURA
#========================================================================================================================#
# moldura da figura
fig, ax = uplt.subplots(figsize=(7,7), tight=True, proj='pcarree')

# data da imagem
datax = f'{anoi}-{mesi}-{diai} {hori}:{mini} - {anof}-{mesf}-{diaf} {horf}:{minf}'

# formatação dos eixos da figura
ax.format(coast=True, borders=True, innerborders=False,
          labels=True, latlines=20, lonlines=20,
          latlim=(latmin_america, latmax_america), lonlim=(lonmin_america, lonmax_america),
          title=f'GOES-{goes_number} GLM Flashes\n{datax} UTC')

# plota flashes: 0-30min
ax.scatter(df_00_to_30min['lon'].values,
           df_00_to_30min['lat'].values,
           transform=ccrs.PlateCarree(),
           marker='o', s=10, facecolor='red', edgecolor='red',
           label=f'0-30min = {df_00_to_30min.shape[0]} Flashes',
           zorder=4)

# plota flashes: 30-60min
ax.scatter(df_30_to_60min['lon'].values,
           df_30_to_60min['lat'].values,
           transform=ccrs.PlateCarree(),
           marker='o', s=10, facecolor='yellow', edgecolor='yellow',
           label=f'30-60min = {df_30_to_60min.shape[0]} Flashes',
           zorder=3)

# plota flashes: 60-90min
ax.scatter(df_60_to_90min['lon'].values,
           df_60_to_90min['lat'].values,
           transform=ccrs.PlateCarree(),
           marker='o', s=10, facecolor='green', edgecolor='green',
           label=f'60-90min = {df_60_to_90min.shape[0]} Flashes',
           zorder=2)

# plota flashes: 90-120min
total = df_00_to_30min.shape[0] + df_30_to_60min.shape[0] + df_60_to_90min.shape[0] + df_90_to_120min.shape[0]
ax.scatter(df_90_to_120min['lon'].values,
           df_90_to_120min['lat'].values,
           transform=ccrs.PlateCarree(),
           marker='o', s=10, facecolor='cyan', edgecolor='cyan',
           label=f'90-120min = {df_90_to_120min.shape[0]} Flashes\nTotal = {total} Flashes',
           zorder=1)

# plota estados
shapefile = list(shpreader.Reader('https://github.com/evmpython/Minicurso_UFMS_SEMADESC_marco_2026/raw/main/01_utils/BR_UF_2019.shp').geometries())
ax.add_geometries(shapefile, ccrs.PlateCarree(), edgecolor='grey', facecolor='none', linewidth=0.7)

# adiciona legenda
ax.legend(ncol=1, loc='lr', frameon=True)

# salva figura
fig.save(f'{dir_output}/Parte_3_G{goes_number}_GLM_flash_por_tempos.png')

In [ ]:
# mostra os dados
df_flash

---
>`EXERCÍCIO DE FIXAÇÃO:`
1.   Mudar a data/horário.
---

#**PARTE 4)**: `Dados da Amazon a cada 20s` - Visualização da Imagem de Vários Arquivos de Relâmpagos num Mapa Interativo

Nesta aula plotaremos os flashes que ocorreram em diferentes intervalos de tempo, só que agora plotaremos os flashes num mapa interativo. A vantagem de utilizar mapas interativos ao ínves de mapas estáticos, é que podemos realizar um zoom numa região e saber a localização onde ocorreu o relâmpago. Para plotar o mapa interativo utilizaremos a bilioteca [Folium](https://python-visualization.github.io/folium/latest/). O Folium é uma biblioteca de código aberto do Python projetada para a visualização de dados geográficos. Sua função principal é simplificar a criação de mapas interativos na web, combinando a facilidade da sintaxe do Python com o poder da biblioteca JavaScript Leaflet.js.


![Texto alternativo](https://github.com/evmpython/Minicurso_UFMS_SEMADESC_marco_2026/blob/main/04_logos/folium.png?raw=true)

Por brevidade e simplificação no processamento dos dados, utilizaremos os seguintes intervalos de tempo:

1.   `0 e 5min:` flashes que ocorreram entre o horário atual do relógio e 5 minutos antes.

2.   `5 e 10min:` flashes que ocorreram entre 5 minutos antes e 10 minutos antes do horário atual do relógio.

3.   `10 e 15min:`  flashes que ocorreram entre 10 minutos antes e 15 minutos antes do horário atual do relógio.

`Em resumo realizaremos o seguinte passo-a-passo:`

1.   **Importação das bibliotecas**
2.   **Criação do diretório de entrada e saída**
3.   **Extrai a data atual do relógio e determina o horário de 15min antes**
4.   **Download dos arquivos do GLM**
5.   **Leitura dos arquivos do GLM**
6.   **Separa os dados de Flash por classe de tempo (0-5min, 5-10min e 10-15min)**
7.   **Plota figura com o ultraplot**
8.   **Salva link do mapa em formato HTML**

In [ ]:
%%time
#========================================================================================================================#
#                                           IMPORTAÇÃO DAS BIBLIOTECAS
#========================================================================================================================#
import folium
import pandas as pd
import numpy as np
import xarray as xr
import glob
from datetime import timedelta, datetime
import os
import boto3
from botocore import UNSIGNED
from botocore.config import Config
import warnings
warnings.filterwarnings('ignore')

#========================================================================================================================#
#                                        CRIA DIRETÓRIO DE ENTRADA E SAÍDA
#========================================================================================================================#
dir_input = "/content/input/Parte_4"; os.makedirs(dir_input, exist_ok=True)
dir_output = "/content/output/Parte_4"; os.makedirs(dir_output, exist_ok=True)

#========================================================================================================================#
#                                          DEFINE A DATA DO ARQUIVO
#========================================================================================================================#
# extrai a data atual do relógio
agora = datetime.now()
anof, mesf, diaf, horf, minf = str(agora.year), str(agora.month).zfill(2), str(agora.day).zfill(2), str(agora.hour).zfill(2), str(agora.minute).zfill(2)

# calcula a data/hora de 15 minutos atrás
data_15min_atras = datetime.now() - timedelta(minutes=15)

# extrai o ano, mês, dia, hora, minuto inicial (ou seja, 15 min antes da data atual)
anoi = str(data_15min_atras.year)
mesi = str(data_15min_atras.month).zfill(2)
diai = str(data_15min_atras.day).zfill(2)
hori = str(data_15min_atras.hour).zfill(2)
mini = str(data_15min_atras.minute).zfill(2)

#========================================================================================================================#
#                                                DOWNLOAD DOS DADOS
#========================================================================================================================#
# coloca a data INICIAL numa string
date_ini = str(datetime(int(anoi),int(mesi),int(diai),int(hori),int(mini)))

# coloca a data FINAL numa string
date_end = str(datetime(int(anof),int(mesf),int(diaf),int(horf),int(minf)))

# data inicial é fixada como a data do loop
date_loop = date_ini

# loop nos arquivos do GLM
while (date_loop < date_end):

    # data
    yyyymmddhhmnss = datetime.strptime(date_loop, '%Y-%m-%d %H:%M:%S').strftime('%Y%m%d%H%M%S')

    # ano, mes, dia, hora, minuto e segundos
    ano = datetime.strptime(date_loop, '%Y-%m-%d %H:%M:%S').strftime('%Y')
    mes = datetime.strptime(date_loop, '%Y-%m-%d %H:%M:%S').strftime('%m')
    dia = datetime.strptime(date_loop, '%Y-%m-%d %H:%M:%S').strftime('%d')
    hor = datetime.strptime(date_loop, '%Y-%m-%d %H:%M:%S').strftime('%H')
    min = datetime.strptime(date_loop, '%Y-%m-%d %H:%M:%S').strftime('%M')
    seg = datetime.strptime(date_loop, '%Y-%m-%d %H:%M:%S').strftime('%S')

    # define se GOES-16 ou GOES-19
    start_g19 = datetime(2025,4,7,0,0)
    imagem_atual = datetime.strptime(yyyymmddhhmnss, '%Y%m%d%H%M%S')
    goes_number = '16' if imagem_atual < start_g19 else '19'
    print(f'DOWNLOAD DO ARQUIVO GLM ===>>> {ano}-{mes}-{dia} {hor}:{min}:{seg}')

    # download o arquivo
    file_glm20s = download_GLM20s_AWS(yyyymmddhhmnss, goes_number, dir_input)

    # incrementa a variável the date_loop
    date_loop = str(datetime.strptime(date_loop, '%Y-%m-%d %H:%M:%S') + timedelta(seconds=20))
    print('\n')

#========================================================================================================================#
#                                            LEITURA DOS ARQUIVOS DE FLASHES
#========================================================================================================================#
# lista dos arquivos
files = sorted(glob.glob(f'{dir_input}/*nc'))

# inicializa os arrays de time, latitude e longitude dos flashes
times_flash = []
lats_flash = np.array([], dtype=np.float32)
lons_flash = np.array([], dtype=np.float32)

# loop nos arquivos
for file in files:

    # leitura do arquivo
    glm_20s = xr.open_dataset(file)

    # appenda as times/lats/longs
    times_flash.extend(glm_20s['flash_time_offset_of_first_event'].values)
    lats_flash = np.append(lats_flash, glm_20s['flash_lat'].values)
    lons_flash = np.append(lons_flash, glm_20s['flash_lon'].values)

    # fecha o arquivo
    glm_20s.close()

# coloca os flashes num dataframe
data_flash = {'time': times_flash, 'lat': lats_flash, 'lon': lons_flash}
df_flash = pd.DataFrame(data_flash)

# tranforma a coluna data para índice do dataframe
df_flash.set_index('time', inplace=True)

# ordena o dataframe
df_flash.sort_index(inplace=True)

#========================================================================================================================#
#                                            SEPARA OS DADOS POR CLASSE DE TEMPO
#========================================================================================================================#
# data atual do relógio
dt_data_ultima = datetime(int(anof), int(mesf), int(diaf), int(horf), int(minf), 0)

# 0-5min: intervalo entre (data_ultima - 5 min) e data_ultima
dt_time_interval_end_0_5min = dt_data_ultima
dt_time_interval_start_0_5min = dt_data_ultima - timedelta(minutes=5)
time_1a = str(dt_time_interval_end_0_5min)
time_1b = str(dt_time_interval_start_0_5min)

# 5-10min: intervalo entre (data_ultima - 10 min) e (data_ultima - 5 min)
dt_time_interval_end_5_10min = dt_time_interval_start_0_5min
dt_time_interval_start_5_10min = dt_data_ultima - timedelta(minutes=10)
time_2a = str(dt_time_interval_end_5_10min)
time_2b = str(dt_time_interval_start_5_10min)

# 10-15min: intervalo entre (data_ultima - 15 min) e (data_ultima - 10 min)
dt_time_interval_end_10_15min = dt_time_interval_start_5_10min
dt_time_interval_start_10_15min = dt_data_ultima - timedelta(minutes=15)
time_3a = str(dt_time_interval_end_10_15min)
time_3b = str(dt_time_interval_start_10_15min)

# seleciona os flashe de cada classe de tempo
df_0_to_5min = df_flash[(df_flash.index > time_1b) & (df_flash.index <= time_1a)]
df_5_to_10min = df_flash[(df_flash.index > time_2b) & (df_flash.index <= time_2a)]
df_10_to_15min = df_flash[(df_flash.index > time_3b) & (df_flash.index <= time_3a)]

#========================================================================================================================#
#                                                 PLOTA FIGURA
#========================================================================================================================#
# dicionário com os dados organizados
dados = {'0-5min': (df_0_to_5min, 'red'),
         '5-10min': (df_5_to_10min, 'blue'),
         '10-15min': (df_10_to_15min, 'green')}

# ordem desejada
ordem_desejada = ['10-15min', '5-10min', '0-5min']

# criar o mapa base
mapa = folium.Map(location=[0,-70], zoom_start=3, tiles='cartodbdark_matter')

# criar grupos de camadas na ordem desejada com contagem no nome
for nome_horario in ordem_desejada:

    # extrai o dataframe e a cor
    df, cor = dados[nome_horario]

    # calcula o total de raios
    total_raios = len(df)

    # cria o nome com contagem
    nome_com_contagem = f'{nome_horario} = {total_raios} Flashes'

    # adiciona o nome do grupo
    grupo = folium.FeatureGroup(name=nome_com_contagem)

    # loop dos flashes daquel intervalo de tempo
    for idx, row in df.iterrows():
        folium.CircleMarker(location=[row["lat"], row["lon"]],
                            radius=3,
                            color=cor,
                            fill=True,
                            fillColor=cor,
                            fillOpacity=0.8,
                            weight=1).add_to(grupo)

    grupo.add_to(mapa)

# adicionar controle de camadas
folium.LayerControl().add_to(mapa)

# cria a data em formato datetime em UTC
data_utc = datetime(int(anof), int(mesf), int(diaf), int(horf), int(minf))

# converte para horário de Brasília (UTC-3)
data_brasilia = data_utc - timedelta(hours=3)

# formata as datas
data_utc_str = data_utc.strftime("%d/%m/%Y %H:%M") + " UTC"
data_brasilia_str = data_brasilia.strftime("%d/%m/%Y %H:%M") + " BRT (Brasília)"

# título com data da última atualização em ambos os horários (UTC e hora local)
title_html = f'''
             <div style="position: relative; text-align: center;">
                 <h3 style="font-size:20px; margin-bottom:2px;"><b>Mapa de Flashes do GLM (GOES-19) para 3 Intervalos de Horários</b></h3>
                 <p style="font-size:14px; color: #888; margin-top:0px;">
                     <i>Última atualização: {data_brasilia_str}</i><br>
                     <span style="font-size:12px;">({data_utc_str})</span>
                 </p>
             </div>
             '''
mapa.get_root().html.add_child(folium.Element(title_html))

# mostra mapa na tela
mapa

Podemos salvar o mapa num link HTML que poderá ser compartilhado e aberto num Browser de internet. Veja o comando abaixo:

In [ ]:
# salva como arquivo HTML
mapa.save(f'{dir_output}/Parte_4_G19_mapa_interativo_flashes.html')

#**PARTE 5)**: `Dados da Amazon a cada 20s` - Acumulando Relâmpagos para um Dia de Dados

Uma tarefa muito comum em meteorologia é querermos plotar um mapa de flashes num dia que teve muitos relâmpagos. Neste caso o mapa terá tantos pontos (cada ponto representando um flash) que ficará uma nuvem de pontos, impossibilitando assim uma visualização detalhada. Para contornar este desafio, realizamos uma interpolação dos relâmpagos numa matriz de dados. Assim, nesta parte da aula iremos baixar os dados de flashes do dia 13 de novembro de 2025 das 03h30min às 0430min e iremos interpolar os relâmpagos para uma grade de 8km de resolução espacial.

## Download dos Dados

Como primeiro passo iremos baixar os dados de 20s de GLM do FTP da Amazon, de forma semelhante ao que fizemos nas partes anteriores desta aula. Para baixar 1 hora de dados demora aproximadamente 5 min.

In [ ]:
%%time
#========================================================================================================================#
#                                          IMPORTAÇÃO DAS BIBLIOTECAS
#========================================================================================================================#
import xarray as xr
import numpy as np
from datetime import timedelta, datetime
import os
import boto3
from botocore import UNSIGNED
from botocore.config import Config
import ultraplot as uplt
import cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader
import glob
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

#========================================================================================================================#
#                                        CRIA DIRETÓRIO DE ENTRADA E SAÍDA
#========================================================================================================================#
dir_input = "/content/input/Parte_5"; os.makedirs(dir_input, exist_ok=True)
dir_output = "/content/output/Parte_5"; os.makedirs(dir_output, exist_ok=True)

#========================================================================================================================#
#                                          DEFINE A DATA DO ARQUIVO
#========================================================================================================================#
# data inicial e final
anoi, mesi, diai, hori, mini = '2025', '11', '13', '03', '30'
anof, mesf, diaf, horf, minf = '2025', '11', '13', '04', '30'

#========================================================================================================================#
#                                            DOWNLOAD DOS DADOS
#========================================================================================================================#
# data inicial e final junta numa string
date_ini = str(datetime(int(anoi),int(mesi),int(diai),int(hori),int(mini)))
date_end = str(datetime(int(anof),int(mesf),int(diaf),int(horf),int(minf)))

# data inicia é fixada como a data do loop
date_loop = date_ini

# loop nos arquivos do GLM
while (date_loop < date_end):

    # data
    yyyymmddhhmnss = datetime.strptime(date_loop, '%Y-%m-%d %H:%M:%S').strftime('%Y%m%d%H%M%S')

    # ano, mes, dia, hora, minuto e segundos
    ano = datetime.strptime(date_loop, '%Y-%m-%d %H:%M:%S').strftime('%Y')
    mes = datetime.strptime(date_loop, '%Y-%m-%d %H:%M:%S').strftime('%m')
    dia = datetime.strptime(date_loop, '%Y-%m-%d %H:%M:%S').strftime('%d')
    hor = datetime.strptime(date_loop, '%Y-%m-%d %H:%M:%S').strftime('%H')
    min = datetime.strptime(date_loop, '%Y-%m-%d %H:%M:%S').strftime('%M')
    seg = datetime.strptime(date_loop, '%Y-%m-%d %H:%M:%S').strftime('%S')

    # define se GOES-16 ou GOES-19
    start_g19 = datetime(2025,4,7,0,0)
    imagem_atual = datetime.strptime(yyyymmddhhmnss, '%Y%m%d%H%M%S')
    goes_number = '16' if imagem_atual < start_g19 else '19'
    print(f'DOWNLOAD DO ARQUIVO GLM ===>>> {ano}-{mes}-{dia} {hor}:{min}:{seg}')

    # download o arquivo
    file_glm20s = download_GLM20s_AWS(yyyymmddhhmnss, goes_number, dir_input)

    # incrementa a variável the date_loop
    date_loop = str(datetime.strptime(date_loop, '%Y-%m-%d %H:%M:%S') + timedelta(seconds=20))
    print('\n')

## Leitura dos dados

In [ ]:
%%time
# lista dos arquivos
files = sorted(glob.glob(f'{dir_input}/*nc'))

# inicializa os arrays de time, latitude e longitude dos flashes
times_flash = []
lats_flash = np.array([], dtype=np.float32)
lons_flash = np.array([], dtype=np.float32)

# loop nos arquivos
for file in files:

    # leitura do arquivo
    glm_20s = xr.open_dataset(file)

    # appenda as times/lats/longs
    times_flash.extend(glm_20s['flash_time_offset_of_first_event'].values)
    lats_flash = np.append(lats_flash, glm_20s['flash_lat'].values)
    lons_flash = np.append(lons_flash, glm_20s['flash_lon'].values)

    # fecha o arquivo
    glm_20s.close()

# coloca os flashes num dataframe
data_flash = {'time': times_flash, 'lat': lats_flash, 'lon': lons_flash}
df_flash = pd.DataFrame(data_flash)

# tranforma a coluna data para índice do dataframe
df_flash.set_index('time', inplace=True)

# ordena o dataframe
df_flash.sort_index(inplace=True)

# mostra os dados
df_flash

## Acumula na Grade

Depois de baixar e ler os dados iremos interpolar cada flash para a grade de 8km. Para isto utilizaremos a função `index` definida no início da aula. Para esta função entraremos com a matriz de latitude e longitude da grade e com a latitude e longitude de cada flash. As características da nossa matriz onde os flashes serão interpolados são:

- Resolução espacial = 8km
- Limites da matriz
    - latmin = -66.56 graus
    - latmax = 66.48 graus
    - lonmin = -141.6 graus
    - lonmax = -8.52 graus

In [ ]:
%%time
# limites da imagem para América
lonmin_america, lonmax_america, latmin_america, latmax_america = -141.6, -8.52, -66.56, 66.48

# espaçamento da grade
delta = 0.08   # grade com 8 km de resolução espacial

# montando a grade
lons = np.arange(lonmin_america, lonmax_america, delta)
lats = np.arange(latmin_america, latmax_america, delta)

# quantidade de pontos para longitude e latitude
nlon = len(lons)
nlat = len(lats)

# transforma de dataframe para numpy.array
flash_lon, flash_lat = df_flash['lon'].values, df_flash['lat'].values

# declara a matriz de relâmpagos
flashes = np.zeros((nlat, nlon))  # Exemplo: flash = np.zeros(qte_linha, qte_coluna)

# loop em cada longitude e latitude da lista
for lonraio, latraio in zip(flash_lon, flash_lat):

    # função que extrai a qual pixel (ou seja, determina as variáveis 'lin' e 'col') aquele relâmpago pertence
    lin, col = index(lons, lats, lonraio, latraio)

    # soma os relâmpagos por pixel
    flashes[lin,col]+=1

# gera netcdf
data_vars = {'flash':(('lat', 'lon'), flashes, {'units': 'ocorrências/km²', 'long_name':'Raios'})}
coords = {'lat': lats, 'lon': lons}
ds = xr.Dataset(data_vars=data_vars, coords=coords)
ds.to_netcdf(f'{dir_output}/Parte_5_flash_glm_{anoi}-{mesi}-{diai}_{hori}{mini}_to_{anof}-{mesf}-{diaf}_{horf}{minf}.nc')

In [ ]:
# mostra o arquivo nectdf que geramos
ds

## Plota Mapa

In [ ]:
#==================================================================================================#
#                                 DEFINIÇÕES DO GRÁFICO
#==================================================================================================#
# cria moldura da figura
fig, ax = uplt.subplots(axheight=6.9, axwidth=6.8, tight=True, proj='pcarree')

# formata os eixos
ax.format(coast=True, borders=True, innerborders=False,
          labels=True, latlines=20, lonlines=20,
          latlim=(latmin_america, latmax_america), lonlim=(lonmin_america, lonmax_america),
          small='15px', large='25px',
          title=f'GOES-{goes_number} GLM Flashes\n{anoi}-{mesi}-{diai} {hori}:{mini} à {anof}-{mesf}-{diaf} {horf}:{minf} UTC')

#==================================================================================================#
#                                       PLOTA FIGURA
#==================================================================================================#
# quantidade de flashes por dia para a área TOTAL da imagem plotada
total = int(ds['flash'].sum(('lat', 'lon')))

# substituir valores "zero" por "NaN"
ds = ds.where(ds != 0, np.nan)

# figura
# aqui dividimos a matriz por 64 para termos a unidade em km2. Lembre-se que
# a matriz de flashes tem tamanho de 8 km x 8 km (ou seja, área de 64 km2).
map1 = ax.contourf(ds['lon'],
                   ds['lat'],
                   ds['flash']/64.,
                   cmap='jet',
                   levels=uplt.arange(0, 5, 1),
                   extend='max')

# total de relâmpagos
ax.text(lonmin_america+0.5, latmin_america+0.9, f'Total Relâmpagos: {int(total)}', color='red', fontsize=15)

# plota contornos dos Estados
shapefile = list(shpreader.Reader('https://github.com/evmpython/Minicurso_UFMS_SEMADESC_marco_2026/raw/main/01_utils/BR_UF_2019.shp').geometries())
ax.add_geometries(shapefile, ccrs.PlateCarree(), edgecolor='grey', facecolor='none', linewidth=0.5, zorder=2)

# barra de cores
fig.colorbar(map1, loc='b', label='relâmpagos/$km^{2}$', ticks=1, ticklabelsize=15, labelsize=15, space=3.1, width=0.3)

# salva figura
fig.save(f'{dir_output}/Parte_5_Fig_1_flash_goes19_{anoi}-{mesi}-{diai} {hori}:{mini}_a_{anof}-{mesf}-{diaf} {horf}:{minf}.jpg', dpi=300)

#**PARTE 6)**: `Dados da Amazon a cada 20s` - Combinação de Imagens de Satélite e Relâmpagos

Os relâmpagos são bons indicadores da intensidade da convecção relacionada as nuvens de tempestades. Desde modo é muito comum em Centros de Monitoramento e Alerta de Tempestades a utilização de mapas combinando a temperatura de brilho do canal infravermelho com os relâmpagos que estão ocorrendo naquele momento. Esta evolução temporal combinada de `Imagem Satélite + Relâmpagos` é essencial para monitorar a conveção e emitir alertas de ocorrência de tempo severo.

Assim, iremos selecionar a imagem de satélite do infravermelho canal 13 do GOES-19 para o dia `13 de novembro de 2025 às 03:30 UTC` quando a cidade de **Campo Grande** foi atingida por intensas tempestades. Como as imagens de satélite são a cada 10min, os relâmpagos serão acumulados num intervalo temporal de 10min para cada imagem de satélite. Então, por exemplo na imagem das 03:30 UTC estará contido os relâmpagos que ocorreram das 03h:30min às 03h:40min UTC.

Parte do processamento que realizaremos será semelhante ao realizado na Aula 1. Ou seja, a parte de realizar o download da imagem de satélite em projeção satélite, transformar para projeção retangular e plotar a imagem do infravermelho. Já para plotar os dados de relâmpagos (*flashes*) do GLM, utilizaremos a função `download_GLM` que esta dentro do código `utilities_goes16e19.py` que baixamos do Github do curso e esta no `/content` da sua sessão do Colab. Nesta função precisaremos apenas indicar a data/hora (formato: ano+mes+dia+hora+minuto+segundos) do arquivo de relâmpago, o satélite que será utilizado (GOES-16 ou GOES-19) e o caminho do diretório onde serão salvos os arquivo do GLM.



![Texto alternativo](https://github.com/evmpython/Minicurso_UFCG_nov_2025/blob/main/logo/geobr_logo_y.png?raw=true)

Além dos dados de relâmpagos, iremos plotar o `shapefile` (contorno) do município de **Campo Grande** no nosso mapa. Para isto utilizaremos a base de shapefiles disponibilizado pela biblioteca [geobr](https://pypi.org/project/geobr/). A geobr disponibiliza o shapefile de todos os muncícipios, regiões metropolitanas, estados, meso e micro regiões do Brasil.



---



Em `resumo` faremos os seguintes procedimentos neste código:

1.   **Importação das bibliotecas**
2.   **Criação do diretório de entrada e saída**
3.   **Download do arquivo da imagem de satélite em projeção satélite**
4.   **Mudança de projeção satélite para retangular**
5.   **Download dos arquivos do GLM dentro de 10min de intervalo temporal**
6.   **Plotagem da imagem de satélite com as informações dos relâmpagos**


In [ ]:
#========================================================================================================================#
#                                          IMPORTAÇÃO DAS BIBLIOTECAS
#========================================================================================================================#
import xarray as xr
import matplotlib.pyplot as plt
from matplotlib import cm
import cartopy, cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader
from datetime import timedelta, datetime
from utilities_goes16e19 import download_CMI, download_GLM, remap, loadCPT
import numpy as np
import os
import pandas as pd
import geobr

#========================================================================================================================#
#                                        CRIA DIRETÓRIO DE ENTRADA E SAÍDA
#========================================================================================================================#
dir_input = "/content/input/Parte_6"; os.makedirs(dir_input, exist_ok=True)
dir_output = "/content/output/Parte_6"; os.makedirs(dir_output, exist_ok=True)

#========================================================================================================================#
#                                               DOWNLOAD DO ARQUIVO
#========================================================================================================================#
# data de processamento: 2025-11-13 às 03:30 UTC
yyyymmddhhmn = '202511130330'

# define o satélite: GOES-16 ou GOES-19
start_g19 = datetime(2025,4,7,0,0)
imagem_atual = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M')
goes_number = '16' if imagem_atual < start_g19 else '19'

# canal do ABI
band = '13'

# download do arquivo (CMI: "Cloud and Moisture Imagery" Product)
file_name = download_CMI(yyyymmddhhmn, band, goes_number, dir_input)

# caminho do arquivo que foi baixado
path = f'{dir_input}/{file_name}.nc'

#========================================================================================================================#
#                                           REPROJEÇÃO SATÉLITE PARA RETANGULAR
#========================================================================================================================#
# área desejada da imagem
lonmin, lonmax, latmin, latmax = -58.5, -50.5, -24.5, -17.0

# coloca os limites da área numa lista
extent = [lonmin, latmin, lonmax, latmax]

# chama a função que faz a reprojeção (file, variable, extent, resolution)
grid = remap(path, 'CMI', extent, 2)

# leitura do dado e transforma para °C
data = grid.ReadAsArray() - 273.15

#========================================================================================================================#
#                                              DOWNLOAD DOS DADOS DO GLM
#========================================================================================================================#
# inicializa os arrays de latitude e longitude dos flashes
lats_flash = np.array([])
lons_flash = np.array([])

# extrai o ano, mês, dia, hora, minuto e segundos
yyyy = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%Y')
mm = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%m')
dd = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%d')
hh = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%H')
mn = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%M')

# data inicial
date_ini = str(datetime(int(yyyy),int(mm),int(dd),int(hh),int(mn)))

# data inicial + 10min
date_end = str(datetime(int(yyyy),int(mm),int(dd),int(hh),int(mn)) + timedelta(minutes=10))
date_loop = date_ini

# Loop nos arquivos do GLM
while (date_loop <= date_end):

    # data
    yyyymmddhhmnss = datetime.strptime(date_loop, '%Y-%m-%d %H:%M:%S').strftime('%Y%m%d%H%M%S') # data no formato: 20251113034000

    # Download o arquivo
    file_glm = download_GLM(yyyymmddhhmnss, goes_number, dir_input)

    # Verifica se o arquivo existe antes de processar
    file_path = f'{dir_input}/{file_glm}.nc'
    if os.path.exists(file_path):

        # leitura do arquivo
        glm_20s = xr.open_dataset(file_path)

        # appenda as lats / longs
        lats_flash = np.append(lats_flash, glm_20s['flash_lat'][:])
        lons_flash = np.append(lons_flash, glm_20s['flash_lon'][:])

        # fecha o arquivo
        glm_20s.close()
    else:
        print(f"Arquivo não encontrado: {file_path}")

    # incrementa a variável the date_loop
    date_loop = str(datetime.strptime(date_loop, '%Y-%m-%d %H:%M:%S') + timedelta(seconds=20))

# coloca os flashes num dataframe
data_flash = {'lat': lats_flash, 'lon': lons_flash}
df = pd.DataFrame(data_flash)

# seleciona os flashes da região de interesse
df_flash_filtered = df[ (df['lon'] > extent[0]) & (df['lon'] < extent[2]) & (df['lat'] > extent[1]) & (df['lat'] < extent[3])]

# transforma o dataframe para array
lons_flash_filtered, lats_flash_filtered = df_flash_filtered['lon'].values, df_flash_filtered['lat'].values

#========================================================================================================================#
#                                                 PLOTA A IMAGEM
#========================================================================================================================#
#----------------------------------------------------------------#
#               Define a configuração do gráfico
#----------------------------------------------------------------#
# tamanho da figura (largura x altura em polegadas)
plt.figure(figsize=(14,11))

# projeção geoestacionária do cartopy
ax = plt.axes(projection=ccrs.PlateCarree())

# define a extensão da imagem
#            lonmin,     lonmax,    latmin,    latmax
img_extent = [extent[0], extent[2], extent[1], extent[3]]

# converte o arquivo CPT para ser usado em Python
cpt = loadCPT('ir.cpt')
colormap = cm.colors.LinearSegmentedColormap('cpt', cpt)

#----------------------------------------------------------------#
#             Plota mapa de temperatura de brilho
#----------------------------------------------------------------#
img = ax.imshow(data,
                origin='upper',
                vmin=-103.0, vmax=84,
                extent=img_extent,
                cmap=colormap)

#----------------------------------------------------------------#
#                   Plota flashes do GLM
#----------------------------------------------------------------#
glm = plt.scatter(lons_flash_filtered,
                  lats_flash_filtered,
                  transform=ccrs.PlateCarree(),
                  marker='o', s=20, facecolor='white', edgecolor='black',
                  linewidth=1,
                  alpha=0.8,
                  zorder=3,
                  label=f'Flashes={str(len(lats_flash_filtered)).zfill(4)}')

#----------------------------------------------------------------#
#                      Plota shapefiles
#----------------------------------------------------------------#
# carrega o shapefiles de todos municípios do Brasil
shapefile_municipios = geobr.read_municipality(year=2020)

# filtra apenas o município de Campo Grande
shapefile_municipio = shapefile_municipios[shapefile_municipios['name_muni'] == 'Campo Grande']

# plota o município de Campo Grande
shapefile_municipio.plot(ax=ax, edgecolor='cyan', facecolor='none', linewidth=2.0, label='Campo Grande', zorder=2)

# carrega o shapefile dos estados brasileiros. Usa a função read_state do geobr para obter os polígonos dos estados
shapefile_estados = geobr.read_state(year=2020)

# filtra os estados dentro da extensão do mapa. Cria uma máscara para selecionar apenas os estados que intersectam com a área do mapa
shapefile_estados_filtrados = shapefile_estados.cx[extent[0]:extent[2], extent[1]:extent[3]]

# plota estados
shapefile_estados_filtrados.plot(ax=ax, edgecolor='yellow', facecolor='none', linewidth=2.0, zorder=3)

#----------------------------------------------------------------#
#           Plota demais formatações do gráfico
#----------------------------------------------------------------#
# define os limites do eixo para a extensão da imagem
ax.set_xlim(extent[0], extent[2])  # lonmin, lonmax
ax.set_ylim(extent[1], extent[3])  # latmin, latmax

# legenda
ax.legend(loc='lower right', ncols=1, facecolor='white', frameon=True)

# linhas costeiras, bordas e linhas de grade do mapa
ax.coastlines(resolution='10m', color='white', linewidth=0.8)
ax.add_feature(cartopy.feature.BORDERS, edgecolor='white', linewidth=0.5)
gl = ax.gridlines(crs=ccrs.PlateCarree(), color='gray', alpha=1.0, linestyle='--', linewidth=0.25, xlocs=np.arange(-180, 180, 3), ylocs=np.arange(-90, 90, 3), draw_labels=True)
gl.top_labels = False
gl.right_labels = False

#----------------------------------------------------------------#
#         Plota barra de cores e títulos da figura
#----------------------------------------------------------------#
# barra de cores
plt.colorbar(img, label='Temperatura de Brilho (°C)', extend='both', orientation='horizontal', pad=0.05, fraction=0.046)

# leitura da data/horário do arquivo NetCDF como uma string
date = (datetime.strptime(xr.open_dataset(path).time_coverage_start, '%Y-%m-%dT%H:%M:%S.%fZ')).strftime('%Y-%m-%d %H:%M UTC')

# título da figura
plt.title(f'GOES-{goes_number} Banda 13 (10.3 µm) + GLM Flashes\nABI: {date}', fontweight='bold', fontsize=10, loc='left')
plt.title(f'GLM: {str(date_ini)} - {str(date_end)}', fontsize="10", loc="right")

#----------------------------------------------------------------#
#                     Salva figura
#----------------------------------------------------------------#
plt.savefig(f'{dir_output}/Parte_6_G{goes_number}_ch{band}_retangular_trealcada_flashes_{date.replace(" ", "_")}.jpg', bbox_inches='tight', dpi=300)

# mostra figura na tela
plt.show()

In [ ]:
# mostra o total de relâmpagos do arquivo
df

In [ ]:
# mostra os relâmpagos selecionados dentro da imagem de satélite
df_flash_filtered

---
>`EXERCÍCIO DE FIXAÇÃO:`
1.   Alterar a data da imagem
---

#**PARTE 7)**: `Dados da Amazon a cada 20s` - Evolução Temporal da Temperatura de Brilho do IR e Relâmpagos

Nesta parte da aula iremos avaliar a evolução temporal da temperatura de bilho das tempestades e os flashes para uma tempestade que atingiu o município de Campo Grande.

Primeiramente plotaremos as imagens de satélite com relâmpagos entre as 03:00 e 05:20 UTC do dia 13 de novembro de 2025. Durante este processamento, para cada imagem iremos extrair a quantidade de relâmpagos que ocorreram dentro de Campo Grande. Depois extrairemos a temperatura média e mínina da tempestade dentro de Campo Grande. E por fim, montaremos uma animação com as imagens e plotaremos num gráfico de linhas a evolução da temperatura média e mínima e dos relâmpagos para a tempestade.


## **Mapa**


Em `resumo` faremos os seguintes procedimentos neste código:

1.   **Importação da bibliotecas**
2.   **Criação do diretório de entrada e saída**
3.   **Define os limites das imagem**
4.   **Carrega os shapefiles dos estados e do município de Campo Grande**
3.   **Processa as imagens**:
        - Extrai a data/horário da imagem
        - Reprojeta de projeção satélite para retangular
        - Realiza o download dos dados do GLM
        - Plota a imagem de satélite com relâmpagos
        - Extrai a temperatura média e mínima dentro de Campo Grande
        - Extrai os relâmpagos de dentro de Campo Grande


In [ ]:
#========================================================================================================================#
#                                          IMPORTAÇÃO DAS BIBLIOTECAS
#========================================================================================================================#
import xarray as xr
import matplotlib.pyplot as plt
from matplotlib import cm
import cartopy, cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader
from datetime import timedelta, datetime
from utilities_goes16e19 import download_CMI, download_GLM, remap, loadCPT
import numpy as np
import os
import pandas as pd
import geopandas as gpd
import salem
import geobr

#========================================================================================================================#
#                                        CRIA DIRETÓRIO DE ENTRADA E SAÍDA
#========================================================================================================================#
dir_input = "/content/input/Parte_7"; os.makedirs(dir_input, exist_ok=True)
dir_output = "/content/output/Parte_7"; os.makedirs(dir_output, exist_ok=True)

#========================================================================================================================#
#                                        DEFINE OS LIMITES DA IMAGEM
#========================================================================================================================#
# canal
band = '13'

# área desejada da imagem
lonmin, lonmax, latmin, latmax = -58.5, -50.5, -24.5, -17.0

# coloca os limites da área numa lista
extent = [lonmin, latmin, lonmax, latmax]

#========================================================================================================================#
#                                              CARREGA SHAPEFILES
#========================================================================================================================#
# carrega o shapefiles de todos municípios do Brasil
shapefile_municipios = geobr.read_municipality(year=2020)

# filtra apenas o município de Campo Grande
shapefile_municipio = shapefile_municipios[shapefile_municipios['name_muni'] == 'Campo Grande']

# carrega o shapefile dos estados brasileiros. Usa a função read_state do geobr para obter os polígonos dos estados
shapefile_estados = geobr.read_state(year=2020)

# filtra os estados dentro da extensão do mapa. Cria uma máscara para selecionar apenas os estados que intersectam com a área do mapa
shapefile_estados_filtrados = shapefile_estados.cx[extent[0]:extent[2], extent[1]:extent[3]]

#========================================================================================================================#
#                                              LOOP DAS IMAGENS
#========================================================================================================================#
# declaração das variáveis da evolução temporal da temperatura e flashes
temp_min_municipio, temp_mean_municipio, flash_total_municipio, time_images = [], [], [], []

# Loop das imagens
for date_image in pd.date_range('202511130300', '202511130520', freq='10min'):

    #--------------------------------------------------------------------------#
    #                          DATA E HORÁRIO
    #--------------------------------------------------------------------------#
    # data
    yyyymmddhhmn = date_image.strftime('%Y%m%d%H%M') # '202511130330'

    # define o satélite: GOES-16 ou GOES-19
    start_g19 = datetime(2025,4,7,0,0)
    imagem_atual = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M')
    goes_number = '16' if imagem_atual < start_g19 else '19'

    # extrai o ano, mês, dia, hora e minuto
    yyyy = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%Y')
    mm = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%m')
    dd = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%d')
    hh = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%H')
    mn = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%M')
    time_images.append(f'{hh}:{mn}')

    print('#=====================================================================================================#')
    print(f'                          PROCESSANDO A IMAGEM = {yyyy}-{mm}-{dd} {hh}{mn} UTC'                       )
    print('#=====================================================================================================#')

    # download do arquivo
    file_name = download_CMI(yyyymmddhhmn, band, goes_number, dir_input)

    # caminho do arquivo que foi baixado
    path = f'{dir_input}/{file_name}.nc'

    #--------------------------------------------------------------------------#
    #                    REPROJETA OS DADOS DO ABI
    #--------------------------------------------------------------------------#
    # chama a função que faz a reprojeção (file, variable, extent, resolution)
    grid = remap(path, 'CMI', extent, 2)

    # leitura do dado e transforma para °C
    data = grid.ReadAsArray() - 273.15

    #--------------------------------------------------------------------------#
    #                       BAIXA OS DADOS DO GLM
    #--------------------------------------------------------------------------#
    # data da imagem atual
    date_ini = str(datetime(int(yyyy),int(mm),int(dd),int(hh),int(mn)))

    # data da imagem atual + 10 min
    date_end = str(datetime(int(yyyy),int(mm),int(dd),int(hh),int(mn)) + timedelta(minutes=10))
    date_loop = date_ini

    # loop de cumulação do GLM
    lats_flash, lons_flash = np.array([]), np.array([])
    while (date_loop <= date_end):

        # data
        yyyymmddhhmnss = datetime.strptime(date_loop, '%Y-%m-%d %H:%M:%S').strftime('%Y%m%d%H%M%S')

        # Download o arquivo
        file_glm = download_GLM(yyyymmddhhmnss, goes_number, dir_input)

        # Verifica se o arquivo existe antes de processar
        file_path = f'{dir_input}/{file_glm}.nc'
        if os.path.exists(file_path):

            # leitura do arquivo
            glm_20s = xr.open_dataset(file_path)

            # appenda as lats / longs
            lats_flash = np.append(lats_flash, glm_20s['flash_lat'][:])
            lons_flash = np.append(lons_flash, glm_20s['flash_lon'][:])

            # fecha o arquivo
            glm_20s.close()
        else:
            print(f"Arquivo não encontrado: {file_path}")

        # incrementa a variável the date_loop
        date_loop = str(datetime.strptime(date_loop, '%Y-%m-%d %H:%M:%S') + timedelta(seconds=20))
    #--------------------------------------------------------------------------#

    # coloca os flashes num dataframe
    data_flash = {'lat': lats_flash, 'lon': lons_flash}
    df = pd.DataFrame(data_flash)

    # seleciona os flahes da região de interesee
    df_flash_filtered = df[ (df['lon'] > lonmin) & (df['lon'] < lonmax) & (df['lat'] > latmin) & (df['lat'] < latmax)]

    # transforma o dataframe para array
    lons_flash_filtered, lats_flash_filtered = df_flash_filtered['lon'].values, df_flash_filtered['lat'].values

    #--------------------------------------------------------------------------#
    #                           PLOTA A IMAGEM
    #--------------------------------------------------------------------------#
    # tamanho da figura (largura x altura em polegadas)
    plt.figure(figsize=(14,11))

    # projeção geoestacionária do cartopy
    ax = plt.axes(projection=ccrs.PlateCarree())

    # converte o arquivo CPT para ser usado em Python
    cpt = loadCPT('ir.cpt')
    cmap = cm.colors.LinearSegmentedColormap('cpt', cpt)

    # plota imagem
    img = ax.imshow(data,
                    origin='upper',
                    vmin=-103.0, vmax=84,
                    extent=[lonmin, lonmax, latmin, latmax],
                    cmap=cmap, alpha=1.0)

    # plota glm
    glm = plt.scatter(lons_flash_filtered,
                      lats_flash_filtered,
                      transform=ccrs.PlateCarree(),
                      marker='o', s=20, facecolor='white', edgecolor='black',
                      linewidth=1, alpha=0.8, zorder=3,
                      label=f'Flashes={str(len(lats_flash_filtered)).zfill(4)}')

    # legenda
    ax.legend(loc='lower right', ncols=1, facecolor='white', frameon=True)

    # linhas costeiras, bordas e linhas de grade do mapa
    gl = ax.gridlines(crs=ccrs.PlateCarree(), color='gray', alpha=1.0, linestyle='--', linewidth=0.25, xlocs=np.arange(-180, 180, 3), ylocs=np.arange(-90, 90, 3), draw_labels=True)
    gl.top_labels = False
    gl.right_labels = False

    # plota o município de Campo Grande
    shapefile_municipio.plot(ax=ax, edgecolor='cyan', facecolor='none', linewidth=2.0, label='Campo Grande', zorder=2)

    # plota estados
    shapefile_estados_filtrados.plot(ax=ax, edgecolor='yellow', facecolor='none', linewidth=2.0, zorder=3)

    # define os limites do eixo para a extensão da imagem
    ax.set_xlim(extent[0], extent[2])  # lonmin, lonmax
    ax.set_ylim(extent[1], extent[3])  # latmin, latmax

    # barra de cores
    plt.colorbar(img, label='Temperatura de Brilho (°C)', extend='both', orientation='horizontal', pad=0.05, fraction=0.046)

    # leitura da data/horário do arquivo NetCDF como uma string
    date = (datetime.strptime(xr.open_dataset(path).time_coverage_start, '%Y-%m-%dT%H:%M:%S.%fZ')).strftime('%Y-%m-%d %H:%M UTC')

    # título da figura
    plt.title(f'GOES-{goes_number} Banda 13 (10.3 µm) + GLM Flashes\nABI: {date}', fontweight='bold', fontsize=10, loc='left')
    plt.title(f'GLM: {str(date_ini)} - {str(date_end)}', fontsize="10", loc="right")

    # salva figura
    plt.savefig(f'{dir_output}/Parte_7_G{goes_number}_ch{band}_retangular_trealcada_flashes_{yyyy}-{mm}-{dd}_{hh}:{mn}_UTC.jpg', bbox_inches='tight', dpi=300)

    #--------------------------------------------------------------------------#
    #   EXTRAI A TEMPERATURA QUE ESTA DENTRO DO MUNICÍPIO DE CAMPO GRANDE
    #--------------------------------------------------------------------------#
    # leitura do arquivo reprojetado
    dataset_ir = xr.open_dataset(f'{dir_input}/{file_name}_ret.nc', mask_and_scale=True).sel(lon=slice(lonmin, lonmax), lat=slice(latmax, latmin))

    # aplica o fator de escala (/10) e transforma para Graus Celsius
    dataset_ir['Band1'] = (dataset_ir['Band1']/10.) - 273.15

    # extrai os valores apenas dentro do município de Campo Grande
    data_ir_municipio = dataset_ir['Band1'].salem.roi(shape=shapefile_municipio)

    # calcula a temperatura mínima e média
    temp_min, temp_mean = float(data_ir_municipio.min(('lon', 'lat'))), float(data_ir_municipio.mean(('lon', 'lat')))

    # appenda as variáveis
    temp_min_municipio.append(temp_min)
    temp_mean_municipio.append(temp_mean)

    #--------------------------------------------------------------------------#
    #    EXTRAI OS FLASHES QUE ESTÃO DENTRO DO MUNICÍPIO DE CAMPO GRANDE
    #--------------------------------------------------------------------------#
    # novo geodataframe
    df_flash_filtered_gpd = gpd.GeoDataFrame(df_flash_filtered.reset_index(),
                                             geometry=gpd.points_from_xy(df_flash_filtered.lon,
                                                                         df_flash_filtered.lat))

    # desigina o systema de coordenadas (CRS)
    df_flash_filtered_gpd.crs = shapefile_municipio.crs

    # aplica a máscara
    df_flash_filtered_gpd_municipio = gpd.overlay(df_flash_filtered_gpd,
                                                  shapefile_municipio,
                                                  how='intersection')

    # appenda os flashes
    flash_total_municipio.append(df_flash_filtered_gpd_municipio.shape[0])
    print('\n')

#--------------------------------------------------------------------------#
#                      INSERE NUM DATAFRAME
#--------------------------------------------------------------------------#
data_ts = {'time': time_images,
           'temp_min_municipio': temp_min_municipio,
           'temp_mean_municipio': temp_mean_municipio,
           'flash_total_municipio': flash_total_municipio}

df_ts = pd.DataFrame(data_ts)

In [ ]:
# flashes
df_flash_filtered

In [ ]:
# mostrando o dataframe com temperaturas e flashes
df_ts

## **Animação das imagens**

- O objetivo aqui é montar uma animação com as imagens de satélites que foram geradas na pasta  `/content/output/Parte_7`. Para isto usaremos a biblioteca [imageio](https://imageio.readthedocs.io/en/stable/).

In [ ]:
# importa bibliotecas
import imageio
import glob

# lista as imagens que serão usadas na animação
files = sorted(glob.glob(f'{dir_output}/*jpg'))

# faz animação
images = []
for file in files:
    images.append(imageio.imread(file))

# salva animação
imageio.mimsave(f'{dir_output}/Parte_7_animacao.gif',
                images,
                duration=300,
                loop=0)

# mostra a animação
print("\nAbrindo o GIF..\n")
from IPython.display import Image
Image(open(f'{dir_output}/Parte_7_animacao.gif','rb').read(), width=600)

##**Evolução Temporal**

In [ ]:
# mostrando os dados que iremos utilizar
df_ts

In [ ]:
#==================================================================================#
#                        CONFIGURAÇÃO INICIAL
#==================================================================================#
# importa biblioteca
import matplotlib.pyplot as plt

# tamanho da figura
fig, ax1 = plt.subplots(figsize=(10, 6))

#==================================================================================#
#             PLOT-1: temperatura no eixo y primério (ax1)
#==================================================================================#
# plota figura
ax1.plot(df_ts['time'].values, df_ts['temp_min_municipio'], marker='o', color='blue', linestyle='--', label='Temp. Mínima')
ax1.plot(df_ts['time'].values, df_ts['temp_mean_municipio'], marker='o', color='black', linestyle='--', label='Temp. Média')

# nomes dos eixos X e Y
ax1.set_xlabel('Tempo (UTC)', color='black', size=15)
ax1.set_ylabel('Temperatura de Brilho ($^o$C)', color='black', size=15)

# configuração dos eixos X e Y
ax1.tick_params(axis='x', labelcolor='black', labelsize=15, rotation=30)
ax1.tick_params(axis='y', labelcolor='black', labelsize=15)

# legenda
ax1.legend(loc='upper right', frameon=False)

#==================================================================================#
#             PLOT-2: flash no eixo y secundário (ax2)
#==================================================================================#
# adiciona eixo adicional
ax2 = ax1.twinx()

# plota figura
ax2.plot(df_ts['time'].values, df_ts['flash_total_municipio'].values, marker='o', color='green', label='Total Flash')

# nome eixo Y
ax2.set_ylabel('Total Flashes (fl/10min)', color='black', size=15)

# configuração do eixo Y
ax2.tick_params(axis='y', labelcolor='black', labelsize=15)

# adiciona legenda
ax2.legend(bbox_to_anchor=(0.975, 0.91), frameon=False)

# título da figura
plt.title(f'Evolução Temporal da Temperatura de Brilho (ABI) e Flashes (GLM)')

#==================================================================================#
#                           SALVA E MOSTRA A FIGURA
#==================================================================================#
# salva a figura
plt.savefig(f'{dir_output}/Parte_7_evolucacao_temporal_temperatura_e_flashes.png', bbox_inches='tight', dpi=300)

# mostra a figura
plt.show()

---
>`EXERCÍCIO DE FIXAÇÃO:`
1.   Alterar o município.
---

#**PARTE 8)**: `Dados do INPE a cada 5min` - Download e Processamento

Até o momento utilizamos os dados do GLM que estão com resolução temporal de 20s. Porém, para análises climatológicas tradicionalmente utilizamos dados interpolados numa grade e que estejam acumulados num intervalo temporal maior como, na escala de minutos, dias e meses. Assim, o CPTEC/INPE processa e disponibiliza os dados  do GLM do [GOES-16](https://ftp.cptec.inpe.br/goes/goes16/goes16_web/glm_acumulado_nc/) e [GOES-19](https://ftp.cptec.inpe.br/goes/goes19/goes19_web/glm_acumulado_nc/) acumulados numa grade de 8km acumulados a cada 5 minutos abrangendo às Américas. No FTP os arquivos estão dispostos em diretórios separados por ano e mês.

Os dados estão salvos em arquivos no formato netCDF como arrays de 1440 latitudes e 1605 longitudes. Os arquivos possuem a seguinte nomenclatura:

- `S11162307_202511130330.nc:`
    - **S11162307:** código do produto. Para o **GOES-16 é S11635949** e **GOES-19 é S11162307**.
    - **202511130330:** data e horário do arquivo no formato AAAAMMDDHHMN. Neste caso ano=2025, mês=11, dia=13, hora=03 e minuto=30.
    - **nc:** formato do dado netCDF.  


Download do arquivo

In [ ]:
# define a data
ano, mes, dia, hor, min = '2025', '11', '13', '03', '30'

# ftp do cptec
ftp = 'http://ftp.cptec.inpe.br/goes/goes19/goes19_web/glm_acumulado_nc/'

# nome do arquivo
filename = f'S11162307_{ano}{mes}{dia}{hor}{min}.nc'

# ftp + nome do arquivo
ftp_filename = f'{ftp}{ano}/{mes}/S11162307_{ano}{mes}{dia}{hor}{min}.nc' # S11162307_202511130330.nc

# cria diretório de entrada onde será salvo o arquivo do CPTEC
dir_input = "/content/input/Parte_8"; os.makedirs(dir_input, exist_ok=True)

# baixando a imagem de satélite
!wget {ftp_filename} -P {dir_input}

Leitura do arquivo

In [ ]:
# leitura do arquivo
glm_5min = xr.open_dataset(f'{dir_input}/{filename}')

In [ ]:
# exibe os dados
glm_5min

Note que o dataset tem `10 variáveis`. Abaixo segue uma explicação detalhada de cada uma:

1. `event:` matriz (time, lat, lon) da quantidade de `eventos`
(event/5minutes/64km2) por ponto de grade no formato float.

2. `group:`  matriz (time, lat, lon) da quantidade de `grupos`
(group/5minutes/64km2) por ponto de grade no formato float.

3. `flash:`  matriz (time, lat, lon) da quantidade de `flash` (flash/5minutes/64km2) por ponto de grade no formato float.

4. `event_energy:` matriz (time, lat, lon) da energia média (Joules) dos `eventos` por ponto de grade no formato float.

5. `group_energy:`  matriz (time, lat, lon) da energia média (Joules) dos `grupos` por ponto de grade no formato float.

6. `flash_energy:`  matriz (time, lat, lon) da energia média (Joules) dos `flash` por ponto de grade no formato float.

7. `duration_flash:` matriz (time, lat, lon) da duração média (Segundos) dos `flash` por ponto de grade no formato float. A duração do flash é calculada através da diferença entre o tempo do último e primeiro evento pertecente ao flash.

8. `event_count:` quantidade de eventos dentro do arquivo, valor inteiro.

9. `group_count:` quantidade de grupos dentro do arquivo, valor inteiro.

10. `flash_count:` quantidade de flashes dentro do arquivo, valor inteiro.

Exibindo os dados

In [ ]:
# mostrando o dataset do valor da quantidade de eventos daquele arquivo
glm_5min['event_count']

In [ ]:
# mostrando o valor da quantidade de EVENTOS daquele arquivo
glm_5min['event_count'].values

In [ ]:
# mostrando o valor da quantidade de GRUPOS daquele arquivo
glm_5min['group_count'].values

In [ ]:
# mostrando o valor da quantidade de FLASH daquele arquivo
glm_5min['flash_count'].values

In [ ]:
# formato dos dados de flash
glm_5min['flash'].shape

In [ ]:
# dados de flash
glm_5min['flash']

In [ ]:
# latitudes
glm_5min['lat']

In [ ]:
# longitudes
glm_5min['lon']

In [ ]:
# mapa simples da quantidade de FLASH
glm_5min['flash'][0, :, :].plot(cmap='jet')

In [ ]:
# mapa simples da ENERGIA do flash
glm_5min['flash_energy'][0, :, :].plot(cmap='jet')

In [ ]:
# mapa simples da DURAÇÃO do flash
glm_5min['duration_flash'][0, :, :].plot(cmap='jet')

Agora que conhecemos os dados de 5min do GLM, iremos gerar alguns mapas e séries temporais. Então bora começar! 🚀

#**PARTE 9)**: `Dados do INPE a cada 5min` - Visualização de Mapa de Relâmpago para um Determinado Dia

Nesta etapa da aula iremos gerar o mapa do `acumulado diário de flash` para as Américas e para o Estado do Mato Grosso do Sul. O objetivo é identificar quais regiões do estado tiveram mais relâmpagos neste dia. Mapas deste tipo são muito importantes para a realização de uma avaliação e inventário dos impactos que uma tempestade tenha provocado num determinado dia no estado do MS. Para isto, iremos realizar esta análise para o dia 13 de novembro de 2025 (mesmo dia que temos utilizado desde o início desta aula do curso).

## Baixando os dados


Os dados serão baixados do [ftp](https://ftp.cptec.inpe.br/goes/goes19/goes19_web/glm_acumulado_nc/) do CPTEC/INPE para o satélite GOES-19. Ao todo serão baixados para o dia 13 de novembro de 2025 `235 arquivos` com frequência temporal de 5min, com duração de download estimada em aproximadamente 5 min.

In [ ]:
%%time
#========================================================================================================================#
#                                          IMPORTAÇÃO DAS BIBLIOTECAS
#========================================================================================================================#
import os
import pandas as pd
from datetime import timedelta, datetime

#========================================================================================================================#
#                                        CRIA DIRETÓRIO DE ENTRADA E SAÍDA
#========================================================================================================================#
dir_input = "/content/input/Parte_9"; os.makedirs(dir_input, exist_ok=True)
dir_output = "/content/output/Parte_9"; os.makedirs(dir_output, exist_ok=True)

#========================================================================================================================#
#                                          DEFINE A DATA INICIAL E FINAL
#========================================================================================================================#
# ano, mês e dia INICIAL
anoi, mesi, diai = 2025, 11, 13

# ano, mês e dia FINAL
anof, mesf, diaf = 2025, 11, 14

#========================================================================================================================#
#                                            DOWNLOAD DOS DADOS
#========================================================================================================================#
# data INICIAL no formato string
date_in = datetime(anoi, mesi, diai)
date_ini = date_in.strftime('%Y%m%d')

# data FINAL no formato string
date_en = datetime(anof, mesf, diaf)
date_end = date_en.strftime('%Y%m%d')

# download dos dados
files_imagens = []
for file in pd.date_range(date_ini, date_end, freq='5min'):

    # extrai ano e mes
    ano = file.strftime('%Y')
    mes = file.strftime('%m')
    dia = file.strftime('%d')
    dia = file.strftime('%d')
    hor = file.strftime('%H')
    min = file.strftime('%M')

    # define se GOES-16 ou GOES-19
    start_g19 = datetime(2025,4,7,0,0)
    imagem_atual = datetime.strptime(str(file), '%Y-%m-%d %H:%M:%S')
    goes_number = '16' if imagem_atual < start_g19 else '19'

    # baixa os dados do GLM
    file_glm_download, codigo_produto = download_GLM5min_CPTEC(ano, mes, dia, hor, min, goes_number, dir_input)

    # salva o nomes das imagens baixadas
    files_imagens.append(file_glm_download)

    print(file, goes_number)

## Leitura dos dados

In [ ]:
%%time
#==================================================================================================#
#                                   IMPORTA BIBLIOTECAS
#==================================================================================================#
import xarray as xr
import glob
import cartopy.io.shapereader as shpreader
import warnings
warnings.filterwarnings("ignore")

#========================================================================================================================#
#                                        CRIA DIRETÓRIO DE ENTRADA E SAÍDA
#========================================================================================================================#
dir_input = "/content/input/Parte_9"; os.makedirs(dir_input, exist_ok=True)
dir_output = "/content/output/Parte_9"; os.makedirs(dir_output, exist_ok=True)

#==================================================================================================#
#                                   DEFINIÇÃO DOS LIMITES DA IMAGEM
#==================================================================================================#
# limites da imagem para o MS
lonmin_MS, lonmax_MS, latmin_MS, latmax_MS = -58.5, -50.5, -24.5, -17.0

# limites da imagem para a América
lonmin_america, lonmax_america, latmin_america, latmax_america = -141.6, -8.52, -66.56, 66.48

# extensão da imagem [min. lon, min. lat, max. lon, max. lat]
extent = [lonmin_america, latmin_america, lonmax_america, latmax_america]

#==================================================================================================#
#                              SHAPEFILE DE MATO GROSSO DO SUL
#==================================================================================================#
shp_MS = list(shpreader.Reader('https://github.com/evmpython/shapefile/raw/main/UFs/MS/MS_UF_2019.shp').geometries())

#==================================================================================================#
#                                   LEITURA DOS DADOS DO GLM-CPTEC
#==================================================================================================#
# lista os arquivos
files = sorted(glob.glob(f'{dir_input}/*_20251113*nc'))

# leitura dos arquivos
glm_5min = xr.open_mfdataset(files,
                             concat_dim = 'time',
                             combine = 'nested').sel(lon=slice(lonmin_america, lonmax_america), lat=slice(latmin_america, latmax_america))

#==================================================================================================#
#                         SOMA OS RELÂMPAGOS PARA CADA DIA
#==================================================================================================#
glm_dia = glm_5min['flash'].sum(dim='time')

In [ ]:
# mostra os dados de 5min
glm_5min

In [ ]:
# mostra o dado diário
glm_dia

## Plota Figura para as `Américas`

In [ ]:
#==================================================================================================#
#                                   IMPORTA BIBLIOTECAS
#==================================================================================================#
import ultraplot as uplt
import cartopy.io.shapereader as shpreader
import cartopy, cartopy.crs as ccrs
import warnings
warnings.filterwarnings("ignore")

#==================================================================================================#
#                              DEFINIÇÕES DO GRÁFICO
#==================================================================================================#
# cria moldura da figura
fig, ax = uplt.subplots(axheight=6.9, axwidth=6.8, tight=True, proj='pcarree')

# formata os eixos
ax.format(coast=True, borders=True, innerborders=False,
          labels=True, latlines=20, lonlines=20,
          latlim=(latmin_america, latmax_america), lonlim=(lonmin_america, lonmax_america),
          small='15px', large='25px',
          title=f'GOES-{goes_number} GLM Flashes: Américas\n2025-11-13')

#==================================================================================================#
#                                       PLOTA FIGURAS
#==================================================================================================#
# quantidade de flashes por dia para a América
total = int(glm_dia[:,:].sum(('lon', 'lat')))

# substituir valores zero por NaN
glm_dia = glm_dia.where(glm_dia != 0, np.nan)

# figura
# aqui dividimos a matriz por 64 para termos a unidade em km2. Lembre-se que
# a matriz de flashes tem tamanho de 8 km x 8 km (ou seja, área de 64 km2).
map1 = ax.contourf(glm_dia['lon'],
                   glm_dia['lat'],
                   glm_dia[:,:]/64.,
                   cmap='jet',
                   levels=uplt.arange(0, 7, 1),
                   extend='max')

# total de relâmpagos
ax.text(lonmin_america+0.5, latmin_america+0.9, f'Total Relâmpagos: {int(total)}', color='red', fontsize=15)

# plota contornos dos Estados
shapefile = list(shpreader.Reader('https://github.com/evmpython/Minicurso_UFMS_SEMADESC_marco_2026/raw/main/01_utils/BR_UF_2019.shp').geometries())
ax.add_geometries(shapefile, ccrs.PlateCarree(), edgecolor='grey', facecolor='none', linewidth=0.5, zorder=2)

# barra de cores
fig.colorbar(map1, loc='b', label='relâmpagos/$km^{2}$*dia', ticks=1, ticklabelsize=15, labelsize=15, space=3.1, width=0.3)

# salva figura
fig.save(f'{dir_output}/Parte_9_Fig_1_relampagos_goes19_2025-11-13_Americas.jpg', dpi=300)

# exibe a figura na tela
uplt.show()

## Plota Figura para o `Mato Grosso do Sul (MS)`

In [ ]:
#==================================================================================================#
#                                   IMPORTA BIBLIOTECAS
#==================================================================================================#
import ultraplot as uplt
import cartopy.io.shapereader as shpreader
import salem
import numpy as np
import warnings
warnings.filterwarnings("ignore")

#==================================================================================================#
#                              DEFINIÇÕES DO GRÁFICO
#==================================================================================================#
# cria moldura da figura
fig, ax = uplt.subplots(axheight=6.9, axwidth=6.8, tight=True, proj='pcarree')

# formata os eixos
ax.format(coast=False, borders=False, innerborders=False,
          labels=True, latlines=2, lonlines=2,
          latlim=(latmin_MS, latmax_MS), lonlim=(lonmin_MS, lonmax_MS),
          small='25px', large='25px',
          title=f'GOES-{goes_number} GLM Flashes: Mato Grosso do Sul\n2025-11-13')

#==================================================================================================#
#                                       PLOTA FIGURAS
#==================================================================================================#
# recorta os dados para MS
glm_dia_MS = glm_dia.sel(lon=slice(lonmin_MS, lonmax_MS), lat=slice(latmin_MS, latmax_MS))

# elimina os dados fora do MS
shapefile_MS = salem.read_shapefile('https://github.com/evmpython/shapefile/raw/main/UFs/MS/MS_UF_2019.shp')
glm_dia_MS = glm_dia_MS.salem.roi(shape=shapefile_MS)

# quantidade de flashes por dia para a área TOTAL da imagem plotada
total = int(glm_dia_MS[:,:].sum(('lon', 'lat')))

# substituir valores zero por NaN
glm_dia_MS = glm_dia_MS.where(glm_dia_MS != 0, np.nan)

# figura
# aqui dividimos a matriz por 64 para termos a unidade em km2. Lembre-se que
# a matriz de flashes tem tamanho de 8 km x 8 km (ou seja, área de 64 km2).
map1 = ax.contourf(glm_dia_MS['lon'],
                   glm_dia_MS['lat'],
                   glm_dia_MS[:,:]/64.,
                   cmap='jet',
                   levels=uplt.arange(0, 7, 1),
                   extend='max')

# total de relâmpagos
ax.text(lonmin_MS+0.1, latmin_MS+0.1, f'Total Relâmpagos: {int(total)}', color='red', fontsize=15)

# plota contorno do MS
shapefile = list(shpreader.Reader('https://github.com/evmpython/shapefile/raw/main/UFs/MS/MS_UF_2019.shp').geometries())
ax.add_geometries(shapefile, ccrs.PlateCarree(), edgecolor='black', facecolor='none', linewidth=1.2, zorder=2)

# barra de cores
fig.colorbar(map1, loc='b', label='relâmpagos/$km^{2}$*dia', ticks=1, ticklabelsize=15, labelsize=15, space=0.8, width=0.3)

# salva figura
fig.save(f'{dir_output}/Parte_9_Fig_2_relampagos_goes19_2025-11-13_MS.jpg', dpi=300)

# exibe a figura na tela
uplt.show()

---
>`EXERCÍCIO DE FIXAÇÃO:`
1.   Mudar o Estado Brasileiro.
---

#**PARTE 10)**: `Dados do INPE a cada 5min` - Visualização de Série Temporal

Além de mapas da densidade de relâmpagos, é comum precisarmos determinar a evolução temporal dos relâmpagos numa região. Por exemplo, saber como foi e evolução temporal a cada 5 min ou a cada hora no dia 13 de novembro de 2025. Assim, nesta última etapa desta aula iremos analisar a série temporal de flash ao longo do dia 13, com frequência temporal de 5min e horária.

## Baixando os dados do GLM

In [ ]:
%%time
#========================================================================================================================#
#                                          IMPORTAÇÃO DAS BIBLIOTECAS
#========================================================================================================================#
import os
import pandas as pd
from datetime import timedelta, datetime
import warnings
warnings.filterwarnings("ignore")

#========================================================================================================================#
#                                        CRIA DIRETÓRIO DE ENTRADA E SAÍDA
#========================================================================================================================#
dir_input = "/content/input/Parte_10"; os.makedirs(dir_input, exist_ok=True)
dir_output = "/content/output/Parte_10"; os.makedirs(dir_output, exist_ok=True)

#========================================================================================================================#
#                                          DEFINE A DATA INICIAL E FINAL
#========================================================================================================================#
# ano, mês e dia INICIAL
anoi, mesi, diai = 2025, 11, 13

# ano, mês e dia FINAL
anof, mesf, diaf = 2025, 11, 14

#========================================================================================================================#
#                                            DOWNLOAD DOS DADOS
#========================================================================================================================#
# data INICIAL no formato string
date_in = datetime(anoi, mesi, diai)
date_ini = date_in.strftime('%Y%m%d')

# data FINAL no formato string
date_en = datetime(anof, mesf, diaf)
date_end = date_en.strftime('%Y%m%d')

# download dos dados
files_imagens = []
for file in pd.date_range(date_ini, date_end, freq='5min'):

    # extrai ano e mes
    ano = file.strftime('%Y')
    mes = file.strftime('%m')
    dia = file.strftime('%d')
    dia = file.strftime('%d')
    hor = file.strftime('%H')
    min = file.strftime('%M')

    # define se GOES-16 ou GOES-19
    start_g19 = datetime(2025,4,7,0,0)
    imagem_atual = datetime.strptime(str(file), '%Y-%m-%d %H:%M:%S')
    goes_number = '16' if imagem_atual < start_g19 else '19'

    # baixa os dados do GLM
    file_glm_download, codigo_produto = download_GLM5min_CPTEC(ano, mes, dia, hor, min, goes_number, dir_input)

    # salva o nomes das imagens baixadas
    files_imagens.append(file_glm_download)

    print(file, goes_number)

## Leitura dos dados

In [ ]:
%%time
#========================================================================================================================#
#                                          IMPORTAÇÃO DAS BIBLIOTECAS
#========================================================================================================================#
import glob
import xarray as xr
import warnings
warnings.filterwarnings("ignore")

#========================================================================================================================#
#                                        CRIA DIRETÓRIO DE ENTRADA E SAÍDA
#========================================================================================================================#
dir_input = "/content/input/Parte_10"; os.makedirs(dir_input, exist_ok=True)
dir_output = "/content/output/Parte_10"; os.makedirs(dir_output, exist_ok=True)

#========================================================================================================================#
#                                           DEFINE LIMITES DA IMAGEM
#========================================================================================================================#
# limites da imagem para Américas
lonmin_america, lonmax_america, latmin_america, latmax_america = -141.6, -8.52, -66.56, 66.48

# limites da imagem para MS
lonmin_MS, lonmax_MS, latmin_MS, latmax_MS = -58.5, -50.5, -24.5, -17.0

#========================================================================================================================#
#                                           LEITURA DOS ARQUIVOS
#========================================================================================================================#
# lista dos arquivos
files = sorted(glob.glob(f'{dir_input}/*_20251113*nc'))

# leitura dos arquivos
glm_5min = xr.open_mfdataset(files,
                             concat_dim = 'time',
                             combine = 'nested').sel(lon=slice(lonmin_MS, lonmax_MS), lat=slice(latmin_MS, latmax_MS))

In [ ]:
# mostra os dados
glm_5min

In [ ]:
# mostra o dado de flash
glm_5min['flash']

In [ ]:
# mostra o dado de time
glm_5min['time']

## Plota Série Temporal: A cada 5min

In [ ]:
# soma os flashes na área
serie_temporal_5min = glm_5min['flash'].sum(('lon', 'lat'))
serie_temporal_5min

In [ ]:
# mostra o time
serie_temporal_5min['time']

In [ ]:
# importa biblioteca
import ultraplot as uplt
import matplotlib.dates as mdates

# cria moldura da figura
fig, ax = uplt.subplots(figsize=(8,5), tight=True)

# plota figura
ax.plot(serie_temporal_5min['time'], serie_temporal_5min, color='red')

# formato dos eixos
data = f'{anoi}-{mesi}-{diai}'

# configura formato do eixo x
ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(mdates.AutoDateLocator())

# plota figura
ax.format(xlabel='Tempo (Hora:Minuto)',
          ylabel='Flashes / 5min',
          ltitle=f'Série Temporal de 5 min de Flashes para o Mato Grosso do Sul\nPeríodo: {data}',
          xrotation=30,
          ytickminor=False,
          large='20px', small='17px')

# salva figura
fig.save(f'{dir_output}/Parte_10_Fig_1_serie_temporal_5min.jpg', dpi=300, bbox_inches='tight')

## Plota Série Temporal: A cada hora

In [ ]:
# reamostra para escala horária
glm_hora = glm_5min['flash'].resample(time='1H').sum()
glm_hora

In [ ]:
# extrai a série temporal (soma os flashes na área)
serie_temporal_hora = glm_hora.sum(('lon', 'lat'))
serie_temporal_hora

In [ ]:
# importa biblioteca
import ultraplot as uplt
import matplotlib.dates as mdates

# cria moldura da figura
fig, ax = uplt.subplots(figsize=(8,5), tight=True)

# plota figura
ax.plot(serie_temporal_hora['time'], serie_temporal_hora, color='red',
        marker='o', markersize=8, markerfacecolor='white', markeredgecolor='bright red',
        markeredgewidth=2)

# formato dos eixos
data = f'{anoi}-{mesi}-{diai}'

# configura formato do eixo x
ax.xaxis.set_major_formatter(mdates.DateFormatter('%H'))
ax.xaxis.set_major_locator(mdates.AutoDateLocator())  # Ajusta automaticamente os ticks

# plota figura
ax.format(xlabel='Hora (UTC)',
          ylabel='Flashes / Hora',
          ltitle=f'Série Temporal Horária de Flashes para o Mato Grosso do Sul\nPeríodo: {data}',
          xrotation=0,
          ytickminor=False,
          large='20px', small='17px')

# salva figura
fig.save(f'{dir_output}/Parte_10_Fig_2_serie_temporal_hora.jpg', dpi=300, bbox_inches='tight')

---
>`EXERCÍCIO DE FIXAÇÃO:`
1.   Mudar a data (ano, mês e dia).
---

# **Parabéns**, você chegou ao final da segunda aula do curso `PyVisSat` !!! 👏 👏 👏

Muito obrigado pela sua participação e nos vamos na próxima aula.

E lembre-se: `Pratique Python, pois Python facilita a nossa vida` !!!

![Texto alternativo](https://github.com/evmpython/Minicurso_UFCG_nov_2025/blob/main/logo/istockphoto-1292897490-612x612.jpg?raw=true)